In [ ]:
"""
2020-2021
Code written (mostly) by Viva Horowitz to plot the modemaps from SIMS data
"""

## initialize

import numpy as np 
import os 
import matplotlib.pyplot as plt 
import seaborn as sns
import pandas as pd 
from myheatmap import myheatmap
import matplotlib as mpl
import scipy as sp
#import cmasher as cmr # required for choice of cyclic colormaps

############### Mac vs Windows ##############
#folder1 = '/Volumes/Aleman-Lab/Group/Projects/GrapheneCoupledResonators/Data/Sample2/2019/2019/08/09/'
#folder2 = '/Volumes/Aleman-Lab/Group/Projects/GrapheneCoupledResonators/Data/Sample2/2019/07/30/'
#folder1 = r'\\cas-fs1.uoregon.edu\Material-Science-Institute\Aleman-Lab\Group\Projects\GrapheneCoupledResonators\Data\Sample2\2019\2019\08\09'
#folder2 = r'\\cas-fs1.uoregon.edu\Material-Science-Institute\Aleman-Lab\Group\Projects\GrapheneCoupledResonators\Data\Sample2\2019\07\30'
folder1 = r'G:\My Drive\Horowitz Research\Oregon\2019-08-09_data_from_Brittany'
folder2 = r'G:\My Drive\Horowitz Research\Oregon\2019-07-30_sample2_data_from_Brittany'

#savefolder = '/Volumes/Aleman-Lab/Group/Papers and Abstracts/Coupled Resonator Spatial Imaging/Figures/python_exports'
savefolder = r'\\cas-fs1.uoregon.edu\Material-Science-Institute\Aleman-Lab\Group\Papers and Abstracts\Coupled Resonator Spatial Imaging\Figures\python_exports'
#############################################

ticklength = 2
tickwidth = .5
cbarticklength = 1

print(mpl.__version__) # version 3 required for cmap='twilight'

sns.set_context('poster') # makes text larger

add_circle = True
plot3d = False
saving = False
verboseplot = False # Plot everything or just the necessities?

## range in which to plot angles (in degrees)
degreerange = (-360,0) # angles should be negative to match simulations.

## number of bins for histograms
numbins = 65

# colormaps: PiYG, vlag_r, bwr_r, RdBu, Seismic_r, twilight & twilight_shifted, cmr.emergency
divergingcmap =  'twilight' # cmr.emergency
cmapforhalf = 'twilight_shifted' # cmr.emergency


# source: https://stackoverflow.com/questions/7404116/defining-the-midpoint-of-a-colormap-in-matplotlib
class MidpointNormalize(mpl.colors.Normalize):
    def __init__(self, vmin, vmax, midpoint=0, clip=False):
        self.midpoint = midpoint
        mpl.colors.Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
        normalized_min = max(0, 1 / 2 * (1 - abs((self.midpoint - self.vmin) / (self.midpoint - self.vmax))))
        normalized_max = min(1, 1 / 2 * (1 + abs((self.vmax - self.midpoint) / (self.midpoint - self.vmin))))
        normalized_mid = 0.5
        x, y = [self.vmin, self.midpoint, self.vmax], [normalized_min, normalized_mid, normalized_max]
        return sp.ma.masked_array(sp.interp(value, x, y))

# make histograms extend to the range set by degreerange    
def includeendpoints(alist):
    blist = list([min(degreerange)])
    blist.extend(bin_centers)
    blist.append(max(degreerange))
    return np.array(blist)

# use myheatmap to plot a masked heatmap.
def maskplot(df, mk, column, uselog=False, cbarlims = None, **kwargs):
    ### cbarlims is not implemented yet.
#    try:
    rlogmkname = column + 'logmk'
    rmkname = column + 'mk'
    df[rlogmkname] = np.log10((df[column])[mk]) 
    df[rmkname] = (df[column])[mk] # python gives a warning but df.loc[:,(column,mk)] didn't work

    if uselog:
        values = rlogmkname
    else:
        values = rmkname
    dfgrid = df.pivot_table(index = 'Y um', columns = 'X um', values = values).sort_index(axis = 0, ascending = False)
    ax,cbar = myheatmap(dfgrid, values, return_cbar = True, **kwargs)
    plt.xlabel(u'$x$ (μm)')
    plt.ylabel(u'$y$ (μm)')
    plt.gca().axis('equal')
    return ax, cbar

In [ ]:
## In case of module not found, run these:

#!pip install hsluv
#!pip install cmasher

In [ ]:
## Create grid of circles
# These circles correspond to the pillars holding the graphene.

### settings for adding grid of circles
pitch = 3.0 # 3 um from one peg to the next
rotationdegrees = -339 # 339 degrees of rotation from straight
radius = .55 # radius of circles to draw in um

xstart = -5.75 
ystart = 10.3

rotation = rotationdegrees * np.pi / (180)
deltax = pitch * np.cos(rotation)
deltay = pitch * np.sin(rotation)

# create grid about origin
xlist = [i*pitch for i in range(-3,6)]
ylist = [i*pitch for i in range(-6,4)]
#display(xlist)

"""    grid = np.meshgrid(xlist,ylist)
print(grid)"""

# rotate grid

# Generate a meshgrid and rotate it by RotRad radians.
# source: https://stackoverflow.com/questions/29708840/rotate-meshgrid-with-numpy
def DoRotation(xspan, yspan, RotRad=0):
    """Generate a meshgrid and rotate it by RotRad radians."""

    # Clockwise, 2D rotation matrix
    RotMatrix = np.array([[np.cos(RotRad),  np.sin(RotRad)],
                          [-np.sin(RotRad), np.cos(RotRad)]])

    x, y = np.meshgrid(xspan, yspan)
    return np.einsum('ji, mni -> jmn', RotMatrix, np.dstack([x, y]))

rotated = DoRotation(xlist,ylist, RotRad=rotation)
#display(rotated)

# translate grid to xstart and ystart
rottrans = rotated.copy()
rottrans[0] = rotated[0]+xstart
rottrans[1] = rotated[1]+ystart

#display(rottrans)

# create tuple list

xs = (rottrans[0]).flatten()
ys = (rottrans[1]).flatten()
coordlist = []
for i in range(len(xs)):
    # only add circle to the coordlist if it's in the mapped region
    if xs[i]>-13.1 and xs[i] < 8.5 and ys[i] > -4 and ys[i] < 17:
        coordlist.append((xs[i], ys[i]))

#display(coordlist)
display(len(coordlist))

# two removed pegs create resonators
coordlist.remove((-4.024362569144298, 6.424154871872494))
coordlist.remove((-0.7990510232365908, 14.826378710347306))


display(len(coordlist))
#display(coordlist)


""" Horrible hack for rotating the image 90 degrees to the right """
def rotatehack(df):
    rotatedf = df.transpose()
    rotatedf.index = -rotatedf.index # I know, it's such a bad hack.
    return rotatedf

# Swap tuple elements in list of tuples 
coordlistrot = [(coord[1], -coord[0]) for coord in coordlist] 

In [ ]:
## Define radians_correction function to correct the phase
# This is based on the time-delay of the experimental setup.

## slope and offset for correction from Three Mass Model LmFit.ipynb
slope =  -7.51633340320429e-07
offset =  67.96713435722867

def radians_correction(Hz):
    omega = Hz * 2 * np.pi
    return slope * omega + offset

print("Approx correction to radians at 15.17 MHz: add", np.round(radians_correction(15.17*1e6),3)) # radians
print("Approx correction to radians at 15.51 MHz: add", np.round(radians_correction(15.51*1e6),3)) # radians

In [ ]:
file = 'PosGU01Array01.00r0.5p3Array01.00r0.5p3Device1.2Drum_15.16MHz_1104325' 
path = os.path.join(folder1,file)
df = pd.read_csv(os.path.join(path, 'Demod1.csv'), skiprows = 2) 
df['X um']=df['Green X']
df['Y um']= df['Green Y']
low_mode_Hz = df.frequency[0]
low_mode_MHz = low_mode_Hz/1e6
print("Correction to radians at low mode,", low_mode_MHz ,"MHz: add", radians_correction(df.frequency[0])) 
# calculate corrected phase
df['phase_cor'] = df.phase + radians_correction(df.frequency[0]) ## correction for radians
df['phase_cor_wrap'] = df.phase_cor % (2 * np.pi) - 2 * np.pi # force angle to be between -2 pi and 0


fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13.23,5))

#plt.suptitle(file + '\n\n');

plt.sca(ax1)
amp1 = df.pivot_table(index = 'Y um', columns = 'X um', values = 'r').sort_index(axis = 0, ascending = False) 
doubleresonatordata = (amp1)
# set white at max, cut colorbar in half
vmin = np.array(doubleresonatordata).min()
vmax = np.array(doubleresonatordata).max()
greennorm = MidpointNormalize(vmin=vmin, vmax=vmax, midpoint=vmax)

myheatmap(doubleresonatordata, 'amplitude', norm = greennorm,
          cmap=cmapforhalf);
plt.xlabel(u'$x$ [μm]')
plt.ylabel(u'$y$ [μm]')
ax1.axis('equal')
ax1.set_title(str(round(df.frequency[0]/1e6,3))+ ' MHz')

if add_circle:
    for coords in coordlist:
        thiscircle = plt.Circle(coords, radius=radius, color='w', fill = None, alpha = .5)
        plt.gca().add_artist(thiscircle)


plt.sca(ax2)
phase1 = df.pivot_table(index = 'Y um', columns = 'X um', values = 'phase_cor_wrap').sort_index(axis = 0, ascending = False) 
myheatmap(phase1, 'phase', vmin=-2*np.pi, vmax = 0, cmap=divergingcmap);
ax2.axis('equal');
plt.xlabel(u'$x$ [μm]')
plt.ylabel(u'$y$ [μm]')

if add_circle:
    for coords in coordlist:
        thiscircle = plt.Circle(coords, radius=radius, color='k', fill = None, alpha = .8)
        plt.gca().add_artist(thiscircle)

ax2.set_title(str(round(df.frequency[0]/1e6,3))+ ' MHz')
plt.tight_layout();


file = 'PosGU01Array01.00r0.5p3Array01.00r0.5p3Device1.2Drum1_15.5MHz105133' 
path = os.path.join(folder1,file)
df = pd.read_csv(os.path.join(path, 'Demod1.csv'), skiprows = 2) 
df['X um']=df['Green X']
df['Y um']= df['Green Y']
high_mode_Hz = df.frequency[0]
high_mode_MHz = high_mode_Hz/1e6
print("Correction to radians at high mode,", high_mode_MHz ,"MHz: add", radians_correction(df.frequency[0])) 
# calculate corrected phase
df['phase_cor'] = df.phase + radians_correction(df.frequency[0]) ## correction for radians
df['phase_cor_wrap'] = df.phase_cor % (2 * np.pi) - 2 * np.pi

fig, (ax3, ax4) = plt.subplots(1, 2, figsize=(13.23,5))

plt.sca(ax3)
amp2 = df.pivot_table(index = 'Y um', columns = 'X um', values = 'r').sort_index(axis = 0, ascending = False) 
data = amp2
# set white at max, cut colorbar in half
vmin = np.array(data).min()
vmax = np.array(data).max()
greennorm = MidpointNormalize(vmin=vmin, vmax=vmax, midpoint=vmax)

myheatmap(data, 'amplitude', norm = greennorm,
          cmap=cmapforhalf);
plt.xlabel(u'$x$ [μm]')
plt.ylabel(u'$y$ [μm]')
ax3.axis('equal');
ax3.set_title(str(round(df.frequency[0]/1e6,3))+ ' MHz')

if add_circle:
    for coords in coordlist:
        thiscircle = plt.Circle(coords, radius=radius, color='w', fill = None, alpha = .5)
        plt.gca().add_artist(thiscircle)

plt.sca(ax4) 
phase2 = df.pivot_table(index = 'Y um', columns = 'X um', values = 'phase_cor_wrap').sort_index(axis = 0, ascending = False) 
myheatmap(phase2, 'phase',vmin=-2*np.pi, vmax=0, cmap=divergingcmap);
plt.xlabel(u'$x$ [μm]')
plt.ylabel(u'$y$ [μm]')
ax4.axis('equal');
ax4.set_title(str(round(df.frequency[0]/1e6,3))+ ' MHz')

if add_circle:
    for coords in coordlist:
        thiscircle = plt.Circle(coords, radius=radius, color='k', fill = None, alpha = .8)
        plt.gca().add_artist(thiscircle)

from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
def add_sizebar(ax, size):
    asb = AnchoredSizeBar(ax.transData,
                          size,
                          str(size) + u' μm',
                          loc='lower right',
                          #bbox_to_anchor=(.5,.5),
                          size_vertical=.4,
                          pad=0.1, borderpad=0.5, sep=5,
                          frameon=False)
    ax.add_artist(asb)

    ## uncommenting the following line will create a sizebar a little bit covering the plot
#add_sizebar(ax4, 10)



plt.tight_layout();

In [ ]:
file = 'PosGU01Array01.00r0.5p3Array01.00r0.5p3Device1.2Drum_15.16MHz_1104325' 
path = os.path.join(folder1,file)
df = pd.read_csv(os.path.join(path, 'Demod1.csv'), skiprows = 2) 
    #df.r = df.r/df.r.max() 
    #print(df) 

df['X um']=df['Green X']
df['Y um']= df['Green Y']
# calculate corrected phase
df['phase_cor'] = df.phase + radians_correction(df.frequency[0]) ## correction for radians
df['phase_cor_wrap'] = df.phase_cor % (2 * np.pi) - 2 * np.pi # wrap to be between -2pi and 0

df.rnorm = df.r/df.r.max()

"""## background threshold in rnorm units
backgthresh1516 = 0.0095
print('Amplitude background threshold at 15.166 MHz is currently set to: ' + str(backgthresh1516) + ' of max amplitude = ' 
      + str(backgthresh1516*df.r.max())
      + ' arb. units')"""
## background threshold in r units
backgthresh = 4e-7
print('Amplitude background threshold is ' + str(backgthresh) + ' arb. units')

# Background plot
#mkbackg1516 = df.rnorm < backgthresh1516; # a mask that selects the (low amplitude) background pixels
mkbackg1516 = df.r < backgthresh

if verboseplot:
    plt.figure()
    maskplot(df, mkbackg1516, 'r', cmap = 'viridis')
    plt.title(file + '\n');
    
print('Mean background amplitude at 15.166 MHz: '
      + str(np.mean(df[mkbackg1516].r)))
print('Stdev background amplitude at 15.166 MHz: '
     + str(np.std(df[mkbackg1516].r)))
print('Max amplitude at 15.166 MHz: ')
print(df.r.max())

    #mk1516 = df.rnorm > backgthresh1516 # a mask that selects the (non-negligible amplitude) foreground pixels
mk1516 = df.r > backgthresh

if verboseplot:
    plt.figure()
    maskplot(df, mk1516, 'r', cmap = 'viridis', uselog=True)
    plt.title(file);
else: 
    print('Turn on verboseplot to see how the masks are defined between foreground and background')

In [ ]:
file = 'PosGU01Array01.00r0.5p3Array01.00r0.5p3Device1.2Drum1_15.5MHz105133'  
path = os.path.join(folder1,file)
df = pd.read_csv(os.path.join(path, 'Demod1.csv'), skiprows = 2) 
#df.r = df.r/df.r.max() 
#print(df) 

df['X um']=df['Green X']
df['Y um']= df['Green Y']
# calculate corrected phase
df['phase_cor'] = df.phase + radians_correction(df.frequency[0]) ## correction for radians
df['phase_cor_wrap'] = df.phase_cor % (2 * np.pi) - 2 * np.pi # wrap to be between -2pi and 0

df.rnorm = df.r/df.r.max()

"""## background threshold in rnorm units
backgthresh1550 = 0.06
print('Amplitude background threshold at 15.51 MHz is currently set to: ' + str(backgthresh1550) + ' of max amplitude = ' 
      + str(backgthresh1550*df.r.max())
      + ' arb. units')"""


# Background plot
#mkbackg1550 = df.rnorm < backgthresh1550; # a mask that selects the (low amplitude) background pixels
mkbackg1550 = df.r < backgthresh

if verboseplot:
    plt.figure()
    maskplot(df, mkbackg1550, 'r', cmap = 'viridis')
    plt.title(file);
print('Mean background amplitude at 15.51 MHz: '
      + str(np.mean(df[mkbackg1550].r)))
print('Stdev background amplitude at 15.51 MHz: '
     + str(np.std(df[mkbackg1550].r)))

#mk1550 = df.rnorm > backgthresh1550 # a mask that selects the (non-negligible amplitude) foreground pixels
mk1550 = df.r > backgthresh

if verboseplot:
    plt.figure()
    maskplot(df, mk1550, 'r', cmap = 'viridis', uselog=True)
    plt.title(file);
else:
    print('Turn on verbose plot to see how the masks are defined.')

In [ ]:
file = 'PosGU01Array01.00r0.5p3Array01.00r0.5p3Device1.2Drum_15.16MHz_1104325' 
path = os.path.join(folder1,file)
df = pd.read_csv(os.path.join(path, 'Demod1.csv'), skiprows = 2) 
#df.r = df.r/df.r.max() 
#print(df) 

verboseplot = False

df['X um']=df['Green X']
df['Y um']= df['Green Y']
# calculate corrected phase
df['phase_cor'] = df.phase + radians_correction(df.frequency[0]) ## correction for radians
df['phase_cor_wrap'] = df.phase_cor % (2 * np.pi) - 2 * np.pi # wrap to be between -2pi and 0

df.rnorm = df.r/df.r.max()
df.phasedeg_uncor = np.rad2deg(df.phase)
df.phasedeg = np.rad2deg(df.phase_cor_wrap)
#display(df.head())
df1516 = df.copy()

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12.8,5))

#plt.suptitle(file + '\n\n');


plt.sca(ax1) 

amp1 = df.pivot_table(index = 'Y um', columns = 'X um', values = 'r').sort_index(axis = 0, ascending = False) 
data = np.log10(amp1)

# set white at max, cut colorbar in half
vmin = np.array(data).min()
vmax = np.array(data).max()
greennorm = MidpointNormalize(vmin=vmin, vmax=vmax, midpoint=vmax)

myheatmap(data, 'log amplitude', norm = greennorm,
          cmap=cmapforhalf);
plt.xlabel(u'$x$ [μm]')
plt.ylabel(u'$y$ [μm]')
ax1.axis('equal')

if add_circle:
    for coords in coordlist:
        thiscircle = plt.Circle(coords, radius=radius, color='k', fill = None, alpha = .8)
        plt.gca().add_artist(thiscircle)

plt.sca(ax2) 
phase1 = df.pivot_table(index = 'Y um', columns = 'X um', values = 'phase_cor_wrap').sort_index(axis = 0, ascending = False) 
myheatmap(phase1, 'phase [rad]', vmin=-2*np.pi, vmax=0, cmap=divergingcmap);
ax2.axis('equal');
plt.xlabel(u'$x$ [μm]')
plt.ylabel(u'$y$ [μm]')
if add_circle:
    for coords in coordlist:
        thiscircle = plt.Circle(coords, radius=radius, color='k', fill = None, alpha = .8)
        plt.gca().add_artist(thiscircle)
        
plt.tight_layout();


"""plt.figure()
maskplot(df, mkbackg, 'phase_cor_wrap', cmap = divergingcmap)
plt.title(file)"""


df.phasedeg_uncor = np.rad2deg(df.phase) ## uncorrected
df.phasedeg = np.rad2deg(df.phase_cor_wrap) ## corrected
mk = mk1516;
### select R12 and R3 from the foreground pixels
maskR12 = (df.phasedeg_uncor[mk] > 0) & (df['Green X'][mk] < 6)
maskR3 = (df.phasedeg_uncor[mk] < 0) & (df['Green Y'][mk] > 0)
print (df.phasedeg[mk][maskR12].mean())
print (df.phasedeg[mk][maskR3].mean())
print("Phase difference (deg):")
print (df.phasedeg[mk][maskR3].mean() - df.phasedeg[mk][maskR12].mean())

if verboseplot: # Phase histogram 1
    plt.figure(figsize=(1,4))
    n, bins, patches = plt.hist(df.phasedeg[mk], bins=numbins,color=('#276419'), orientation='horizontal', range=degreerange);
    plt.ylabel('Phase (deg)')
    #plt.xlabel('# pixels')
    plt.yticks([df.phasedeg[mk][maskR12].mean() ,0, df.phasedeg[mk][maskR3].mean()])
    plt.ylim(degreerange)
    ax = plt.gca()
    ax.yaxis.set_label_position("right")
    ax.yaxis.tick_right()
    ## color bins
    bin_centers = 0.5 * (bins[:-1] + bins[1:])
    bin_centers = includeendpoints(bin_centers) # the endpoints are set by degreerange.
    # scale values to interval [0,1]
    col = bin_centers - min(bin_centers)
    col /= max(col)
    cm = plt.cm.get_cmap(divergingcmap)
    for c, p in zip(col, patches):
        plt.setp(p, 'facecolor', cm(c))

else:
    print('Turn on verbose plot to see histogram of masked phase data')



if verboseplot:
    plt.figure()
    plt.title(file)
    maskplot(df[mk], maskR3, 'phase_cor_wrap' )    

    plt.figure()
    plt.title(file)
    maskplot(df[mk], maskR12, 'phase_cor_wrap' )
else:
    print('Turn on verbose plot to see the R3 phase and the R12 phase heatmaps (masked)')

print('Mean, first peak:')
print (df.phasedeg[mk][maskR12].mean())
print("Standard dev:")
print (df.phasedeg[mk][maskR12].std())
print("Standard error:")
sterr = df.phasedeg[mk][maskR12].std()/np.sqrt(len(df.phasedeg[mk][maskR12]))
print(sterr)
print("Second peak mean:")
print (df.phasedeg[mk][maskR3].mean())
print("Standard dev:")
print (df.phasedeg[mk][maskR3].std())
print("Stderr:")
stderr2= df.phasedeg[mk][maskR3].std()/np.sqrt(len(df.phasedeg[mk][maskR3]))
print(stderr2)
print("Phase difference (deg): " + str(df.phasedeg[mk][maskR3].mean() - df.phasedeg[mk][maskR12].mean()) + " ± " + str(np.sqrt(sterr**2+stderr2**2)));

"""
plt.figure()
y1 = df.pivot_table(index = 'Y um', columns = 'X um', values = 'y').sort_index(axis = 0, ascending = False) 

myheatmap(y1, 'y') 
#          cmap=divergingcmap);
plt.xlabel(u'$x$ [μm]')
plt.ylabel(u'$y$ [μm]')
plt.gca().axis('equal')

if add_circle:
    for coords in coordlist:
        thiscircle = plt.Circle(coords, radius=radius, color='w', fill = None, alpha = .5)
        plt.gca().add_artist(thiscircle)


plt.figure()
x1 = df.pivot_table(index = 'Y um', columns = 'X um', values = 'x').sort_index(axis = 0, ascending = False) 

myheatmap(x1, 'x') 
#          cmap=divergingcmap);
plt.xlabel(u'$x$ [μm]')
plt.ylabel(u'$y$ [μm]')
plt.gca().axis('equal')

if add_circle:
    for coords in coordlist:
        thiscircle = plt.Circle(coords, radius=radius, color='k', fill = None, alpha = .8)
        plt.gca().add_artist(thiscircle)


plt.figure()
myheatmap(-np.log(-x1), '-log -x', 
          cmap='viridis');
plt.xlabel(u'$x$ [μm]')
plt.ylabel(u'$y$ [μm]')
plt.gca().axis('equal')

if add_circle:
    for coords in coordlist:
        thiscircle = plt.Circle(coords, radius=radius, color='k', fill = None, alpha = .8)
        plt.gca().add_artist(thiscircle)

plt.figure()
myheatmap(np.log(x1), 'log x', 
          cmap='viridis');
plt.xlabel(u'$x$ [μm]')
plt.ylabel(u'$y$ [μm]')
plt.gca().axis('equal')

if add_circle:
    for coords in coordlist:
        thiscircle = plt.Circle(coords, radius=radius, color='k', fill = None, alpha = .8)
        plt.gca().add_artist(thiscircle)
"""
        
file = 'PosGU01Array01.00r0.5p3Array01.00r0.5p3Device1.2Drum1_15.5MHz105133' 
path = os.path.join(folder1,file)
df = pd.read_csv(os.path.join(path, 'Demod1.csv'), skiprows = 2) 
#df.r = df.r/df.r.max() 
#print(df) 
df['X um']=df['Green X']
df['Y um']= df['Green Y']
# calculate corrected phase
df['phase_cor'] = df.phase + radians_correction(df.frequency[0]) ## correction for radians
df['phase_cor_wrap'] = df.phase_cor % (2 * np.pi) - 2 * np.pi # wrap to be between -2pi and 0

df1550 = df.copy()


fig, (ax3, ax4) = plt.subplots(1, 2, figsize=(13.23,5))

plt.sca(ax3)
amp2 = df.pivot_table(index = 'Y um', columns = 'X um', values = 'r').sort_index(axis = 0, ascending = False) 
data = np.log(amp2)
# set white at max, cut colorbar in half
vmin = np.array(data).min()
vmax = np.array(data).max()
greennorm = MidpointNormalize(vmin=vmin, vmax=vmax, midpoint=vmax)

myheatmap(data, 'log amplitude', norm = greennorm,
          cmap=cmapforhalf);
plt.xlabel(u'$x$ [μm]')
plt.ylabel(u'$y$ [μm]')
ax3.axis('equal');

if add_circle:
    for coords in coordlist:
        thiscircle = plt.Circle(coords, radius=radius, color='k', fill = None, alpha = 1)
        plt.gca().add_artist(thiscircle)

plt.sca(ax4)
phase2 = df.pivot_table(index = 'Y um', columns = 'X um', values = 'phase_cor_wrap').sort_index(axis = 0, ascending = False) 
myheatmap(phase2, 'phase [rad]',vmin=-2*np.pi, vmax=0, cmap=divergingcmap);
plt.xlabel(u'$x$ [μm]')
plt.ylabel(u'$y$ [μm]')
ax4.axis('equal');

if add_circle:
    for coords in coordlist:
        thiscircle = plt.Circle(coords, radius=radius, color='k', fill = None, alpha = .8)
        plt.gca().add_artist(thiscircle)

plt.tight_layout();


# Phase histogram 2
df.phasedeg_uncor = np.rad2deg(df.phase)
df.phasedeg = np.rad2deg(df.phase_cor_wrap)
df.rnorm = df.r/df.r.max()
print("rmax = " + str(df.r.max()))
maskR2 = (df.phasedeg_uncor[mk1550] > 0) & (df['Green Y'][mk1550] > 7.0) & (df['Green X'][mk1550] < 0.0) & (df['Green X'][mk1550] > -10.5)
maskR1 = (df.phasedeg_uncor[mk1550] < 0) & (df['Green X'][mk] > -10)
print (df.phasedeg[mk1550][maskR2].mean())
print (df.phasedeg[mk1550][maskR1].mean())
print("Phase difference (deg):")
print (df.phasedeg[mk1550][maskR1].mean() - df.phasedeg[mk1550][maskR2].mean())


if verboseplot:
    plt.figure(figsize=(1,4))
    #mk1550 = df.rnorm > backgthresh1550;
    n, bins, patches = plt.hist(df.phasedeg[mk1550], bins=numbins,color=('#276419'),orientation='horizontal', range=degreerange);
    plt.ylabel('Phase (deg)')
    plt.title('Histogram for thresholded data (15.5 MHz)')
    #plt.xlabel('# pixels')
    ## use uncorrected phase for calculating the mask

    plt.yticks([df.phasedeg[mk1550][maskR2].mean(), 0,df.phasedeg[mk1550][maskR1].mean() ])
    plt.ylim(degreerange)
    ax = plt.gca()
    ax.yaxis.set_label_position("right")
    ax.yaxis.tick_right()
    ## color bins
    bin_centers = 0.5 * (bins[:-1] + bins[1:])
    bin_centers=includeendpoints(bin_centers) # the endpoints are set by degreerange.
    # scale values to interval [0,1]
    col = bin_centers - min(bin_centers)
    col /= max(col)
    cm = plt.cm.get_cmap(divergingcmap)
    for c, p in zip(col, patches):
        plt.setp(p, 'facecolor', cm(c))
else:
    print('Turn on verboseplot for histogram of phase (from thresholded 15.5 MHz data)')
    
"""plt.figure()
maskplot(df, mk1550, 'phase_cor_wrap',cmap='viridis')    
plt.title(file)

plt.figure()
maskplot(df[mk1550], maskR2, 'phase_cor_wrap',cmap='viridis')
plt.title(file + 'resonator 2')

plt.figure()
maskplot(df1516, mk1550, 'phase_cor_wrap',cmap='viridis' )
plt.title('15.166 MHz, foreground of 15.51 MHz')

plt.figure()
maskplot(df1516[mk1550], maskR1, 'phase_cor_wrap',cmap='viridis')
plt.title('15.166 MHz, resonator 1')

plt.figure()
maskplot(df1516[mk1550], maskR2, 'phase_cor_wrap',cmap='viridis')
plt.title('15.166 MHz, resonator 2')


"""
#df1516[mk][maskR2]

"""
plt.figure()
x1 = df.pivot_table(index = 'Y um', columns = 'X um', values = 'x').sort_index(axis = 0, ascending = False) 

myheatmap(x1, 'x', 
          cmap=divergingcmap);
plt.xlabel(u'$x$ [μm]')
plt.ylabel(u'$y$ [μm]')
plt.gca().axis('equal')

if add_circle:
    for coords in coordlist:
        thiscircle = plt.Circle(coords, radius=radius, color='k', fill = None, alpha = .8)
        plt.gca().add_artist(thiscircle)
        

plt.figure()
myheatmap(np.log(-x1), 'log -x', 
          cmap='viridis');
plt.xlabel(u'$x$ [μm]')
plt.ylabel(u'$y$ [μm]')
plt.gca().axis('equal')

if add_circle:
    for coords in coordlist:
        thiscircle = plt.Circle(coords, radius=radius, color='k', fill = None, alpha = .8)
        plt.gca().add_artist(thiscircle)


plt.figure()
myheatmap(np.log(x1), 'log x', 
          cmap='viridis');
plt.xlabel(u'$x$ [μm]')
plt.ylabel(u'$y$ [μm]')
plt.gca().axis('equal')

if add_circle:
    for coords in coordlist:
        thiscircle = plt.Circle(coords, radius=radius, color='k', fill = None, alpha = .8)
        plt.gca().add_artist(thiscircle)
""";


In [ ]:
# Print out averages as approximations for the Z vectors

print('None of these numbers are quite right for the Z vectors. These are not based on paraboloid fits, and the masks are large')
A1 = np.mean((df1516[mk1550])[maskR1].r);
A1avgbigmask1517 = A1
A1stderr = np.std((df1516[mk1550])[maskR1].r) / np.sqrt(len(df1516[mk1550][maskR1].r));
A1uncbigmask1517 = A1stderr
A1maxbigmask1517 = np.max((df1516[mk1550])[maskR1].r)
print('resonator 1 average amplitude at 15.166 MHz: ' + str(A1) + ' ± ' + str(A1stderr));
print('resonator 1 max amplitude at 15:16 MHz: '
     + str(np.max((df1516[mk1550])[maskR1].r)))
print('resonator 1 UNCORRECTED average phase at 15.166 MHz: ' 
      + str(np.mean((df1516[mk1550])[maskR1].phase)) + ' rad')
print('resonator 1 corrected average phase at 15.166 MHz: ' 
      + str(np.mean((df1516[mk1550])[maskR1].phase_cor)) + ' rad')
unc1_uncor = np.std((df1516[mk1550])[maskR1].phase)/np.sqrt(len(df1516[mk1550][maskR1].phase))
uncphase1bigmask1517corrad = np.std(df1516[mk1550][maskR1].phase_cor)/np.sqrt(len(df1516[mk1550][maskR1].phase_cor))
uncphase1bigmask1517cordeg = uncphase1bigmask1517corrad * 180/np.pi;
phase1bigmask1517corrad = np.mean((df1516[mk1550])[maskR1].phase_cor);
phase1bigmask1517cordeg =  phase1bigmask1517corrad * 180/np.pi ;                    
print('resonator 1 UNCORRECTED average phase at 15.166 MHz: ' 
      + str(np.mean((df1516[mk1550])[maskR1].phase) * 180/np.pi) 
      + ' ± ' 
      + str( unc1_uncor * 180/np.pi ) 
      + ' deg');
print('resonator 1 average phase at 15.166 MHz: ' 
      + str(phase1bigmask1517cordeg) 
      + ' ± ' 
      + str( uncphase1bigmask1517cordeg ) 
      + ' deg');

A2 = np.mean((df1516[mk1550])[maskR2].r);
A2stderr = np.std(df1516[mk1550][maskR2].r) / np.sqrt(len(df1516[mk1550][maskR2].r))
A2avgbigmask1517 = A2
A2uncbigmask1517 = A2stderr
A2maxbigmask1517 = np.max(df1516[mk1550][maskR2].r)
print('resonator 2 average amplitude at 15.166 MHz: ' 
      + str(A2)
      + ' ± '
      + str(A2stderr ))
print('resonator 2 max amplitude at 15.166 MHz: '
     + str(np.max(df1516[mk1550][maskR2].r)))
print('max A2 / max A1: '
     + str((np.max(df1516[mk1550][maskR2].r))/ (np.max(df1516[mk1550][maskR1].r))))
print('resonator 2 UNCORRECTED average phase at 15.166 MHz: ' 
      + str(np.mean(df1516[mk1550][maskR2].phase))+ ' rad')
print('resonator 2 average phase at 15.166 MHz: ' 
      + str(np.mean(df1516[mk1550][maskR2].phase_cor))+ ' rad')
unc2_uncor = np.std(df1516[mk1550][maskR2].phase)/(np.sqrt(len(df1516[mk1550][maskR2].phase)))
uncphase2bigmask517corrad = np.std(df1516[mk1550][maskR2].phase_cor)/(np.sqrt(len(df1516[mk1550][maskR2].phase_cor)))
uncphase2bigmask1517cordeg = uncphase2bigmask517corrad * 180/np.pi
print('resonator 2 UNCORRECTED average phase at 15.166 MHz: ' 
      + str(np.mean(df1516[mk1550][maskR2].phase) * 180/np.pi)
      + ' ± '
      + str(unc2_uncor * 180/np.pi)
      + ' deg')

phase2bigmask1517corrad = np.mean(df1516[mk1550][maskR2].phase_cor)
phase2bigmask1517cordeg = phase2bigmask1517corrad* 180/np.pi
print('resonator 2 average phase at 15.166 MHz: ' 
      + str(phase2bigmask1517cordeg)
      + ' ± '
      + str(uncphase2bigmask1517cordeg)
      + ' deg')

print('UNCORRECTED Phase difference $\delta_{12}$ at 15.166 MHz: ' 
     + str((np.mean(df1516[mk1550][maskR2].phase) - np.mean(df1516[mk1550][maskR1].phase)) * 180/np.pi)
     + ' ± '
     + str(np.sqrt(unc1_uncor**2 + unc2_uncor**2) * 180/np.pi)
     + ' deg\n')
print('CORRECTED Phase difference $\delta_{12}$ at 15.166 MHz: ' 
     + str((np.mean(df1516[mk1550][maskR2].phase_cor) - np.mean(df1516[mk1550][maskR1].phase_cor)) * 180/np.pi)
     + ' ± '
     + str(np.sqrt(uncphase1bigmask1517corrad**2 + uncphase2bigmask517corrad**2) * 180/np.pi)
     + ' deg\n')

A3  = np.mean(df1516[mk1516][maskR3].r) 
A3stderr = np.std(df1516[mk1516][maskR3].r) / np.sqrt(len(df1516[mk1516][maskR3].r))
A3avgbigmask1517 = A3
A3uncbigmask1517 = A3stderr
A3maxbigmask1517 = np.max(df1516[mk1516][maskR3].r)
print('resonator 3 average amplitude at 15.166 MHz: ' 
      + str(A3)
      + ' ± '
      + str(A3stderr ));
print('resonator 3 max amplitude at 15.166 MHz: '
     + str(np.max(df1516[mk1516][maskR3].r)));
print('max A3 / max A1: '
     + str((np.max(df1516[mk1516][maskR3].r))/ (np.max(df1516[mk1550][maskR1].r))))
print('max A3 / max A2: '
     + str((np.max(df1516[mk1516][maskR3].r))/ (np.max(df1516[mk1550][maskR2].r))))
print('resonator 3 UNCORRECTED average phase at 15.166 MHz: ' 
      + str(np.mean(df1516[mk1516][maskR3].phase))+ ' rad')
print('resonator 3 corrected average phase at 15.166 MHz: ' 
      + str(np.mean(df1516[mk1516][maskR3].phase_cor))+ ' rad')
phase3bigmask1517corrad = np.mean(df1516[mk1516][maskR3].phase_cor);
phase3bigmask1517cordeg =  phase3bigmask1517corrad * 180/np.pi
unc3_uncor = np.std(df1516[mk1516][maskR3].phase)/(np.sqrt(len(df1516[mk1516][maskR3].phase)))
uncphase3bigmask1517corrad = np.std(df1516[mk1516][maskR3].phase_cor)/(np.sqrt(len(df1516[mk1516][maskR3].phase_cor)))
uncphase3bigmask1517cordeg = uncphase3bigmask1517corrad * 180/np.pi;
print('resonator 3 UNCORRECTED average phase at 15.166 MHz: ' 
      + str(np.mean(df1516[mk1516][maskR3].phase) * 180/np.pi)
      + ' ± '
      + str(unc3_uncor * 180/np.pi)
      + ' deg\n')
print('resonator 3 corrected average phase at 15.166 MHz: ' 
      + str(phase3bigmask1517cordeg)
      + ' ± '
      + str(uncphase3bigmask1517cordeg )
      + ' deg\n')

A1avgbigmask1551 = np.mean(df1550[mk1550][maskR1].r)
A1maxbigmask1551 = np.max(df1550[mk1550][maskR1].r)
A1uncbigmask1551 = np.std(df1550[mk1550][maskR1].r) / np.sqrt(len(df1550[mk1550][maskR1].r))
print('resonator 1 average amplitude at 15.51 MHz: ' 
      + str(A1avgbigmask1551) + ' ± ' 
      + str(A1uncbigmask1551))
print('resonator 1 max amplitude at 15.51 MHz: '
     + str(A1maxbigmask1551))
print('resonator 1 UNCORRECTED average phase at 15.51 MHz: ' 
      + str(np.mean(df1550[mk1550][maskR1].phase)) + ' rad')
print('resonator 1 corrected average phase at 15.51 MHz: ' 
      + str(np.mean(df1550[mk1550][maskR1].phase_cor)) + ' rad')
unc1_uncor = np.std(df1550[mk1550][maskR1].phase)/np.sqrt(len(df1550[mk1550][maskR1].phase))
uncphase1bigmask1551corrad = np.std(df1550[mk1550][maskR1].phase_cor)/np.sqrt(len(df1550[mk1550][maskR1].phase_cor))
uncphase1bigmask1551cordeg = uncphase1bigmask1551corrad * 180/np.pi
print('resonator 1 UNCORRECTED average phase at 15.51 MHz: ' 
      + str(np.mean(df1550[mk1550][maskR1].phase) * 180/np.pi) 
      + ' ± ' 
      + str( unc1_uncor * 180/np.pi ) 
      + ' deg')
phase1bigmask1551corrad =  np.mean(df1550[mk1550][maskR1].phase_cor);
phase1bigmask1551cordeg = phase1bigmask1551corrad * 180/np.pi
print('resonator 1 corrected average phase at 15.51 MHz: ' 
      + str(phase1bigmask1551cordeg ) 
      + ' ± ' 
      + str( uncphase1bigmask1551cordeg ) 
      + ' deg')

A2avgbigmask1551 = np.mean(df1550[mk1550][maskR2].r)
A2uncbigmask1551 = np.std(df1550[mk1550][maskR2].r) / np.sqrt(len(df1550[mk1550][maskR2].r))
A2maxbigmask1551 = np.max(df1550[mk1550][maskR2].r)
print('resonator 2 average amplitude at 15.51 MHz: ' 
      + str(np.mean(df1550[mk1550][maskR2].r))
      + ' ± '
      + str(np.std(df1550[mk1550][maskR2].r) / np.sqrt(len(df1550[mk1550][maskR2].r))))
print('resonator 2 max amplitude at 15.51 MHz: '
     + str(np.max(df1550[mk1550][maskR2].r)))
print('resonator 2 UNCORRECTED average phase at 15.51 MHz: ' 
      + str(np.mean(df1550[mk1550][maskR2].phase))+ ' rad')
print('resonator 2 corrected average phase at 15.51 MHz: ' 
      + str(np.mean(df1550[mk1550][maskR2].phase_cor))+ ' rad')
phase2bigmask1551corrad = np.mean(df1550[mk1550][maskR2].phase_cor)
phase2bigmask1551cordeg = phase2bigmask1551corrad * 180/np.pi
unc2_uncor = np.std(df1550[mk1550][maskR2].phase)/(np.sqrt(len(df1550[mk1550][maskR2].phase)))
uncphase2bigmask1551corrad = np.std(df1550[mk1550][maskR2].phase_cor)/(np.sqrt(len(df1550[mk1550][maskR2].phase_cor)))
uncphase2bigmask1551cordeg = uncphase2bigmask1551corrad * 180/np.pi
print('resonator 2 UNCORRECTED average phase at 15.51 MHz: ' 
      + str(np.mean(df1550[mk1550][maskR2].phase) * 180/np.pi)
      + ' ± '
      + str(unc2_uncor * 180/np.pi)
      + ' deg')
print('resonator 2 CORRECTED average phase at 15.51 MHz: ' 
      + str(phase2bigmask1551cordeg)
      + ' ± '
      + str(uncphase2bigmask1551cordeg)
      + ' deg')

print('UNCORRECTED Phase difference $\delta_{12}$ at 15.51 MHz: ' 
     + str((np.mean(df1550[mk1550][maskR2].phase) - np.mean(df1550[mk1550][maskR1].phase)) * 180/np.pi)
     + ' ± '
     + str(np.sqrt(uncphase1bigmask1551corrad**2 + uncphase2bigmask1551corrad**2) * 180/np.pi)
     + ' deg\n')
print('Corrected Phase difference $\delta_{12}$ at 15.51 MHz: ' 
     + str((np.mean(df1550[mk1550][maskR2].phase_cor) - np.mean(df1550[mk1550][maskR1].phase_cor)) * 180/np.pi)
     + ' ± '
     + str(np.sqrt(uncphase1bigmask1551corrad**2 + uncphase2bigmask1551corrad**2) * 180/np.pi)
     + ' deg\n')

A3avgbigmask1551 = np.mean(df1550[mk1516][maskR3].r)
A3uncbigmask1551 = np.std(df1550[mk1516][maskR3].r) / np.sqrt(len(df1550[mk1516][maskR3].r))
A3maxbigmask1551 = np.max(df1550[mk1516][maskR3].r)
print('resonator 3 average amplitude at 15.51 MHz: ' 
      + str(np.mean(df1550[mk1516][maskR3].r))
      + ' ± '
      + str(np.std(df1550[mk1516][maskR3].r) / np.sqrt(len(df1550[mk1516][maskR3].r))) )
print('resonator 3 max amplitude at 15.51 MHz: '
     + str(np.max(df1550[mk1516][maskR3].r)))
print('resonator 3 UNCORRECTED average phase at 15.51 MHz: ' 
      + str(np.mean(df1550[mk1516][maskR3].phase))+ ' rad')
print('resonator 3 corrected average phase at 15.51 MHz: ' 
      + str(np.mean(df1550[mk1516][maskR3].phase_cor))+ ' rad')
phase3bigmask1551corrad = np.mean(df1550[mk1516][maskR3].phase_cor)
phase3bigmask1551cordeg = phase3bigmask1551corrad * 180/np.pi
unc3_uncor = np.std(df1550[mk1516][maskR3].phase)/(np.sqrt(len(df1550[mk1516][maskR3].phase)))
uncphase3bigmask1551corrad = np.std(df1550[mk1516][maskR3].phase_cor)/(np.sqrt(len(df1550[mk1516][maskR3].phase_cor)))
uncphase3bigmask1551cordeg = uncphase3bigmask1551corrad * 180/np.pi
print('resonator 3 UNCORRECTED average phase at 15.51 MHz: ' 
      + str(np.mean(df1550[mk1516][maskR3].phase) * 180/np.pi)
      + ' ± '
      + str(unc3_uncor * 180/np.pi)
      + ' deg')
print('resonator 3 corrected average phase at 15.51 MHz: ' 
      + str(phase3bigmask1551cordeg)
      + ' ± '
      + str(uncphase3bigmask1551cordeg)
      + ' deg')

In [ ]:
mk = mk1516;
df = df1516;
df.phasedeg_uncor = np.rad2deg(df.phase)
df.phasedeg = np.rad2deg(df.phase_cor_wrap)

if verboseplot:
    plt.figure(figsize=(1,4))

    plt.hist(df.phasedeg[mk], bins=numbins,color=('k'),orientation='horizontal', histtype='step', range=degreerange);
    n, bins, patches = plt.hist(df.phasedeg[mk], bins=numbins, orientation='horizontal', range=degreerange);
    #plt.ylabel('Phase (deg)')
    #plt.xlabel('# pixels')
    print ('Resonators12 phase ' + str(df.phasedeg[mk][maskR12].mean()))
    print ('Resonator 3 phase ' + str(df.phasedeg[mk][maskR3].mean()))
    plt.yticks([df.phasedeg[mk][maskR12].mean() ,0, df.phasedeg[mk][maskR3].mean()])
    plt.ylim(degreerange)
    ax = plt.gca()
    ax.yaxis.set_label_position("right")
    ax.yaxis.tick_right()
    print("Phase difference (deg):")
    print (df.phasedeg[mk][maskR3].mean() - df.phasedeg[mk][maskR12].mean())

    ## color bins, https://stackoverflow.com/questions/23061657/plot-histogram-with-colors-taken-from-colormap
    bin_centers = 0.5 * (bins[:-1] + bins[1:])
    bin_centers=includeendpoints(bin_centers)
    # scale values to interval [0,1]
    col = bin_centers - min(bin_centers)
    col /= max(col)
    cm = plt.cm.get_cmap(divergingcmap)
    for c, p in zip(col, patches):
        plt.setp(p, 'facecolor', cm(c))
    sns.despine(left=True, right=True, bottom=True)    
    

mk = mk1550;
df = df1550;
df.phasedeg_uncor = np.rad2deg(df.phase)
df.phasedeg = np.rad2deg(df.phase_cor_wrap)

if verboseplot:      
    plt.figure(figsize=(1,4))
    plt.hist(df.phasedeg[mk], bins=numbins,color=('k'),orientation='horizontal', histtype='step', range=degreerange);
    n, bins, patches = plt.hist(df.phasedeg[mk], bins=numbins,orientation='horizontal', range=degreerange);
    #plt.ylabel('Phase (deg)')
    #plt.xlabel('# pixels')
    print ('Resonator 2 phase ' + str(df.phasedeg[mk][maskR2].mean()))
    print ('Resonator 1 phase ' + str(df.phasedeg[mk][maskR1].mean()))
    plt.yticks([df.phasedeg[mk][maskR2].mean(), 0,df.phasedeg[mk][maskR1].mean() ])
    plt.ylim(degreerange)
    ax = plt.gca()
    ax.yaxis.set_label_position("right")
    ax.yaxis.tick_right()
    print("Phase difference (deg):")
    print (df.phasedeg[mk][maskR1].mean() - df.phasedeg[mk][maskR2].mean())
    ## color bins, https://stackoverflow.com/questions/23061657/plot-histogram-with-colors-taken-from-colormap
    bin_centers = 0.5 * (bins[:-1] + bins[1:])
    bin_centers=includeendpoints(bin_centers)
    # scale values to interval [0,1]
    col = bin_centers - min(bin_centers)
    col /= max(col)
    cm = plt.cm.get_cmap(divergingcmap)
    for c, p in zip(col, patches):
        plt.setp(p, 'facecolor', cm(c))
    sns.despine(left=True, right=True, bottom=True)

In [ ]:
if verboseplot:
    mk = mk1516;
    df = df1516;

    plt.figure(figsize=(1,4))

    plt.hist(df.phasedeg[mk], bins=numbins,color=('k'),orientation='horizontal', histtype='step', range=degreerange);
    n, bins, patches = plt.hist(df.phasedeg[mk], bins=numbins, orientation='horizontal', range=degreerange);
    #plt.ylabel('Phase (deg)')
    #plt.xlabel('# pixels')
    print ('Resonators12 phase ' + str(df.phasedeg[mk][maskR12].mean()))
    print ('Resonator 3 phase ' + str(df.phasedeg[mk][maskR3].mean()))
    plt.yticks([df.phasedeg[mk][maskR12].mean() ,0, df.phasedeg[mk][maskR3].mean()])
    plt.ylim(degreerange)
    ax = plt.gca()
    ax.yaxis.set_label_position("right")
    ax.yaxis.tick_right()
    print("Phase difference (deg):")
    print (df.phasedeg[mk][maskR3].mean() - df.phasedeg[mk][maskR12].mean())

    ## color bins, https://stackoverflow.com/questions/23061657/plot-histogram-with-colors-taken-from-colormap
    bin_centers = 0.5 * (bins[:-1] + bins[1:])
    bin_centers=includeendpoints(bin_centers)
    # scale values to interval [0,1]
    col = bin_centers - min(bin_centers)
    col /= max(col)
    cm = plt.cm.get_cmap(divergingcmap)
    for c, p in zip(col, patches):
        plt.setp(p, 'facecolor', cm(c))
    sns.despine(left=True, right=True, bottom=True)    


    mk = mk1550;
    df = df1550;

    plt.figure(figsize=(1,4))
    plt.hist(df.phasedeg[mk], bins=numbins,color=('k'),orientation='horizontal', histtype='step', range=degreerange);
    n, bins, patches = plt.hist(df.phasedeg[mk], bins=numbins,orientation='horizontal', range=degreerange);
    #plt.ylabel('Phase (deg)')
    #plt.xlabel('# pixels')
    print ('Resonator 2 phase ' + str(df.phasedeg[mk][maskR2].mean()))
    print ('Resonator 1 phase ' + str(df.phasedeg[mk][maskR1].mean()))
    plt.yticks([df.phasedeg[mk][maskR2].mean(), 0,df.phasedeg[mk][maskR1].mean() ])
    plt.ylim(degreerange)
    ax = plt.gca()
    ax.yaxis.set_label_position("right")
    ax.yaxis.tick_right()
    print("Phase difference (deg):")
    print (df.phasedeg[mk][maskR1].mean() - df.phasedeg[mk][maskR2].mean())
    ## color bins, https://stackoverflow.com/questions/23061657/plot-histogram-with-colors-taken-from-colormap
    bin_centers = 0.5 * (bins[:-1] + bins[1:])
    bin_centers=includeendpoints(bin_centers)
    # scale values to interval [0,1]
    col = bin_centers - min(bin_centers)
    col /= max(col)
    cm = plt.cm.get_cmap(divergingcmap)
    for c, p in zip(col, patches):
        plt.setp(p, 'facecolor', cm(c))
    sns.despine(left=True, right=True, bottom=True)

In [ ]:
### 2021-08-20 goal: plot Amplitude vector as a set of points with error bars

## set values
if False:
    # option 1: values from big mask (not best choice: ask Benjamin for paraboloid fits)
    A1low = A1maxbigmask1517 
    A2low = A2maxbigmask1517
    A3low = A3maxbigmask1517
    A1lowerr = A1uncbigmask1517
    A2lowerr = A2uncbigmask1517
    A3lowerr = A3uncbigmask1517
    print('15.17 MHz, data from big mask');
else:
    # option 2: values from Benjamin
    A1low = 22e-6
    A2low = 37e-6
    A3low = 1.6e-6 
    A1lowerr = 1e-6
    A2lowerr = 2e-6
    A3lowerr = 0.2e-6
    print('15.17 MHz, data from paraboloid fits')

# colors
R1color = '#2A4F98'
R2color = '#E4A52D'
R3color = '#B4237B'

horiz = [1,1,1]
Alowvector = [A1low,A2low,A3low]
Alowerrvector = [A1lowerr,A2lowerr,A3lowerr]

## plot
if verboseplot:
    sns.set_context('paper') # makes text small
    fig = plt.figure(figsize=(1,2))
    plt.errorbar(x=horiz, y = Alowvector, yerr = Alowerrvector,
                 ecolor = 'k',
                 markerfacecolor  = 'k',
                 markeredgewidth  = 0,
                 ls='None', marker = '.', 
                 markersize = 5)
    #             ecolor = [R1color,R2color,R3color],

    loampax = plt.gca()
    loampax.set_yscale('log')
    #loampax.set_xlabel ="Resonator number"
    loampax.set_ylabel="Amplitude (arb. units)"
    loampax.set_ylim(ymin=1e-7)

    loampax.annotate('$A_1$', (horiz[0],A1low),color=R1color, va='center', xytext=(5, 0), textcoords='offset points',)
    loampax.annotate('$A_2$', (horiz[1],A2low), color=R2color, va='center', xytext=(5, 0), textcoords='offset points',)
    loampax.annotate('$A_3$', (horiz[2],A3low), color=R3color, va='center', xytext=(5, 0), textcoords='offset points',)
    sns.despine()

    """n = ["R1", 'R2', 'R3']

    for i, txt in enumerate(n):
        loampax.annotate(txt, (Alowvector[i], horiz[i]))"""

    sns.set_context('poster') # makes text small again going forward

In [ ]:
### 2021-08-20 goal: plot Amplitude vector as a set of points with error bars

if False:
    # option 1: values from big mask (not best choice: ask Benjamin for paraboloid fits)
    A1hi = A1maxbigmask1551
    A2hi = A2maxbigmask1551
    A3hi = A3maxbigmask1551 
    A1hierr = A1uncbigmask1551
    A2hierr = A2uncbigmask1551
    A3hierr = A3uncbigmask1551
    print('15.51 MHz, data from big mask');
else:
    # option 2: values from Benjamin
    A1hi = 7.0e-6
    A2hi = 2.0e-6
    A3hi = 0 
    A1hierr = 0.1e-6
    A2hierr = 0.2e-6
    A3hierr = 0
    print('15.51 MHz, data from paraboloid fits');

horiz = [1,1,1]
Ahivector = [A1hi,A2hi,A3hi]
Ahierrvector = [A1hierr,A2hierr,A3hierr]

verboseplot = True
if verboseplot:
    sns.set_context('paper') # makes text small

    fig = plt.figure(figsize=(1,2))
    plt.errorbar(x=horiz, y = Ahivector, yerr = Ahierrvector,
                 ecolor = 'k',
                 markerfacecolor  = 'k',
                 markeredgewidth  = 0,
                 ls='None', marker = '.', 
                 markersize = 5)
    #             ecolor = [R1color,R2color,R3color],

    hiampax = plt.gca()
    hiampax.set_yscale('log')
    #hiampax.set_xlabel ="Resonator number"
    hiampax.set_ylabel="Amplitude (arb. units)"
    hiampax.set_ylim(ymin=1e-7)

    hiampax.annotate('$A_1$', (horiz[0],A1hi),color=R1color, va='center', xytext=(5, 0), textcoords='offset points',)
    hiampax.annotate('$A_2$', (horiz[1],A2hi), color=R2color, va='center', xytext=(5, 0), textcoords='offset points',)
    #hiampax.annotate('$A_3$', (horiz[2],A3hi), color=R3color, va='center', xytext=(5, 0), textcoords='offset points',)
    sns.despine()

    """n = ["R1", 'R2', 'R3']

    for i, txt in enumerate(n):
        hiampax.annotate(txt, (Ahivector[i], horiz[i]))"""

    sns.set_context('poster') # makes text large again going forward

In [ ]:
### 2021-08-21 goal: plot phase vector as a set of points with error bars

if True:
    # option 1: values from big mask (not best choice)
    d1hi = phase1bigmask1551cordeg
    d2hi = phase2bigmask1551cordeg
    d3hi = phase3bigmask1551cordeg 
    d1hierr = uncphase1bigmask1551cordeg
    d2hierr = uncphase2bigmask1551cordeg
    d3hierr = uncphase3bigmask1551cordeg
else:
    # option 2: values from Benjamin
    d1hi = 0
    d2hi = 0
    d3hi = 0 
    d1hierr = 0.1e-6
    d2hierr = 0.2e-6
    d3hierr = 0

wrapping = True # wrap to degreerange
if wrapping:
    d1hi = d1hi % 360 - 360
    d2hi = d2hi % 360 - 360
    d3hi = d3hi % 360 - 360

horiz = [1,1,1]
dhivector = [d1hi,d2hi,d3hi]
dhierrvector = [d1hierr,d2hierr,d3hierr]


verboseplot=True
if verboseplot:
    sns.set_context('paper') # makes text small

    fig = plt.figure(figsize=(1,2))
    plt.errorbar(x=horiz, y = dhivector, yerr = dhierrvector,
                 ecolor = 'k',
                 markerfacecolor  = 'k',
                 markeredgewidth  = 0,
                 ls='None', marker = '.', 
                 markersize = 5)
    #             ecolor = [R1color,R2color,R3color],

    hiphaseax = plt.gca()
    hiphaseax.yaxis.set_major_locator(plt.MultipleLocator(90))
    #hiphaseax.set_yscale('log')
    #hiphaseax.set_xlabel ="Resonator number"
    hiphaseax.set_ylabel="Amplitude (arb. units)"
    if wrapping:
        hiphaseax.set_ylim(degreerange)

    hiphaseax.annotate('$\delta_1$', (horiz[0],d1hi),color=R1color, va='center', xytext=(5, 0), textcoords='offset points',)
    hiphaseax.annotate('$\delta_2$', (horiz[1],d2hi), color=R2color, va='center', xytext=(5, 0), textcoords='offset points',)
    hiphaseax.annotate('$\delta_3$', (horiz[2],d3hi), color=R3color, va='center', xytext=(5, 0), textcoords='offset points',)
    sns.despine()

    """n = ["R1", 'R2', 'R3']

    for i, txt in enumerate(n):
        hiphaseax.annotate(txt, (Ahivector[i], horiz[i]))"""

    sns.set_context('poster') # makes text large again going forward

In [ ]:
### 2021-08-21 goal: plot phase vector as a set of points with error bars

if True:
    # option 1: values from big mask (not best choice)
    d1low = phase1bigmask1517cordeg
    d2low = phase2bigmask1517cordeg
    d3low = phase3bigmask1517cordeg 
    d1lowerr = uncphase1bigmask1517cordeg
    d2lowerr = uncphase2bigmask1517cordeg
    d3lowerr = uncphase3bigmask1517cordeg
else:
    # option 2: values from Benjamin
    d1low = 0
    d2low = 0
    d3low = 0 
    d1lowerr = 0.1e-6
    d2lowerr = 0.2e-6
    d3lowerr = 0

wrapping = True # wrap to degreerange
if wrapping:
    d1low = d1low % 360 - 360
    d2low = d2low % 360 - 360
    d3low = d3low % 360 - 360

horiz = [1,1,1]
dlowvector = [d1low,d2low,d3low]
dlowerrvector = [d1lowerr,d2lowerr,d3lowerr]


verboseplot=True
if verboseplot:
    sns.set_context('paper') # makes text small

    fig = plt.figure(figsize=(1,2))
    plt.errorbar(x=horiz, y = dlowvector, yerr = dlowerrvector,
                 ecolor = 'k',
                 markerfacecolor  = 'k',
                 markeredgewidth  = 0,
                 ls='None', marker = '.', 
                 markersize = 5)
    #             ecolor = [R1color,R2color,R3color],

    lowphaseax = plt.gca()
    lowphaseax.yaxis.set_major_locator(plt.MultipleLocator(90))
    #lowphaseax.set_yscale('log')
    #lowphaseax.set_xlabel ="Resonator number"
    lowphaseax.set_ylabel="Amplitude (arb. units)"
    if wrapping:
        lowphaseax.set_ylim(degreerange)

    lowphaseax.annotate('$\delta_1$', (horiz[0],d1low),color=R1color, va='center', xytext=(5, 2), textcoords='offset points',)
    lowphaseax.annotate('$\delta_2$', (horiz[1],d2low), color=R2color, va='center', xytext=(5, -2), textcoords='offset points',)
    lowphaseax.annotate('$\delta_3$', (horiz[2],d3low), color=R3color, va='center', xytext=(5, 0), textcoords='offset points',)
    sns.despine()

    """n = ["R1", 'R2', 'R3']

    for i, txt in enumerate(n):
        lowphaseax.annotate(txt, (Alowvector[i], horiz[i]))"""

    sns.set_context('poster') # makes text large again going forward

In [ ]:
sns.set_context('paper') # makes text sized for paper

#saving = True
add_circle= True
rotating = True
#divergingcmap = 'twilight_shifted'
ampcmap = 'Greys' #'magma'
halfaxiscmap = False # slightly broken on the high frequency
includetitles = True

file = 'PosGU01Array01.00r0.5p3Array01.00r0.5p3Device1.2Drum_15.16MHz_1104325' 
path = os.path.join(folder1,file)
df = pd.read_csv(os.path.join(path, 'Demod1.csv'), skiprows = 2) 
#df.r = df.r/df.r.max() 
#print(df) 

df['X um']=df['Green X']
df['Y um']= df['Green Y']
# calculate corrected phase
df['phase_cor'] = df.phase + radians_correction(df.frequency[0]) ## correction for radians
df['phase_cor_wrap'] = df.phase_cor % (2 * np.pi) - 2 * np.pi # wrap to be between -2pi and 0

df.phasedeg_uncor = np.rad2deg(df.phase)
df.phasedeg = np.rad2deg(df.phase_cor_wrap)
df.rnorm = df.r/df.r.max()

widths = [2.6,.7,2.6,.7]
heights = [1, 1]
gs_kw = dict(width_ratios=widths, height_ratios=heights)

## (2,2) size.   #3.37, 2.8 is tight; 3.15 height gives space for scale bar.
sty={'figsize':(5.1,2.9), "constrained_layout":True, "gridspec_kw": gs_kw}

## (1,2) size
#sty={'figsize':(3.37,1.43), "constrained_layout":True}

fig, ((ax1, loampax, ax2, hiampax),(ax3, phist1, ax4, phist2)) = plt.subplots(2,4, **sty)

## label text for scalebar
tenum = u'\u200c          10 μm'

#plt.suptitle(file + '\n\n');
plt.sca(ax1) 

amp1 = df.pivot_table(index = 'Y um', columns = 'X um', values = 'r').sort_index(axis = 0, ascending = False) 
data = np.log10(amp1)
if rotating:
    data = rotatehack(data)
# set white at max, cut colorbar in half
vmin = np.array(data).min()
vmax = np.array(data).max()
greennorm = MidpointNormalize(vmin=vmin, vmax=vmax, midpoint=vmax)

if halfaxiscmap:
    axx,cbar = myheatmap(data,  norm = greennorm, return_cbar=True,
              cmap=cmapforhalf);
else:
    axx,cbar = myheatmap(data,   return_cbar=True,
              cmap=ampcmap);
plt.xlabel('')
plt.ylabel('')
ax1.axis('equal')
topx = data.columns.max()
plt.xticks([topx-10, topx])
plt.gca().set_xticklabels([tenum, ''])
plt.yticks([]) # remove y ticks, since we can use the xticks alone on a square plot
cbar1 = cbar

# decrease number of ticks on colorbar
"""tick_locator = mpl.ticker.MaxNLocator(nbins=4)
cbar.locator = tick_locator
cbar.update_ticks()"""
#cbar.set_ticks([-11, -16])
cbar.set_ticks([-7, -6,-5])
cbar.ax.set_yticklabels(['$10^{-7}$', '$10^{-6}$','$10^{-5}$']) ## make SURE this line is with the one above it
ylims = plt.ylim()
xlims = plt.xlim()

if add_circle:
    if rotating:
        coordinateslist = coordlistrot
    else:
        coordinateslist = coordlist
    for coords in coordinateslist:
        thiscircle = plt.Circle(coords, radius=radius, color='k', fill = None, alpha = 1, lw=0.5,)#linestyle='dashed')
        plt.gca().add_artist(thiscircle)
        
plt.sca(loampax)        
plt.errorbar(x=horiz, y = Alowvector, yerr = Alowerrvector,
             ecolor = 'k',
             markerfacecolor  = 'k',
             markeredgewidth  = 0,
             ls='None', marker = '.', 
             markersize = 5)
#             ecolor = [R1color,R2color,R3color],

loampax = plt.gca()
loampax.set_yscale('log')
#loampax.set_xlabel ="Resonator number"
loampax.set_ylabel="Amplitude (arb. units)"
loampax.set_ylim(ymin=10**vmin, ymax = 10**vmax)
#loampax.axis('off')

loampax.annotate('$A_1$', (horiz[0],A1low),color=R1color, va='center', xytext=(5, -2), textcoords='offset points',)
loampax.annotate('$A_2$', (horiz[1],A2low), color=R2color, va='center', xytext=(5, 2), textcoords='offset points',)
loampax.annotate('$A_3$', (horiz[2],A3low), color=R3color, va='center', xytext=(5, 0), textcoords='offset points',)
sns.despine()

plt.sca(ax3) 
phase1 = df.pivot_table(index = 'Y um', columns = 'X um', values = 'phase_cor_wrap').sort_index(axis = 0, ascending = False) 
data = phase1
if rotating:
    data = rotatehack(data)
axx, cbar = myheatmap(data, return_cbar=True,  vmin=-2*np.pi, vmax=0, cmap=divergingcmap);
ax2.axis('equal');
plt.xlabel('')
plt.ylabel('')
#topx = amp1.columns.max()
plt.xticks([topx-10, topx])
plt.gca().set_xticklabels([tenum, ''])
cbar.ax.yaxis.set_ticks_position('left')
cbar.set_ticks([-2*np.pi,-np.pi,0])
cbar.ax.set_yticklabels(['$-$2\u03C0','$-$\u03C0','0'],ha='right')  # make sure this line is joined with the one above it.
cbar.ax.yaxis.set_tick_params(pad=1)  # your number may vary
plt.yticks([]) # remove y ticks, since we can use the xticks alone on a square plot
cbar2 = cbar

if add_circle:
    if rotating:
        coordinateslist = coordlistrot
    else:
        coordinateslist = coordlist
    for coords in coordinateslist:
        thiscircle = plt.Circle(coords, radius=radius, color='k', fill = None, alpha = .8, lw=0.5,)#linestyle='dashed')
        plt.gca().add_artist(thiscircle)
        
plt.sca(phist1) # Phase histogram 1
mk = mk1516;
plt.hist(df.phasedeg[mk], bins=numbins,color=('k'),orientation='horizontal', 
         histtype='step', range=degreerange, linewidth = 0.5);
n, bins, patches = plt.hist(df.phasedeg[mk], bins=numbins, orientation='horizontal', range=degreerange);
#plt.ylabel('Phase (deg)')
#plt.xlabel('# pixels')
print ('Resonators12 phase ' + str(df.phasedeg[mk][maskR12].mean()))
print ('Resonator 3 phase ' + str(df.phasedeg[mk][maskR3].mean()))
plt.yticks([df.phasedeg[mk][maskR12].mean() ,0, df.phasedeg[mk][maskR3].mean()])
plt.ylim(degreerange)
ax = plt.gca()
ax.yaxis.set_label_position("right")
ax.yaxis.tick_right()
print("Phase difference (deg):")
print (df.phasedeg[mk][maskR3].mean() - df.phasedeg[mk][maskR12].mean())

## color bins, https://stackoverflow.com/questions/23061657/plot-histogram-with-colors-taken-from-colormap
bin_centers = 0.5 * (bins[:-1] + bins[1:])
bin_centers=includeendpoints(bin_centers)
# scale values to interval [0,1]
col = bin_centers - min(bin_centers)
col /= max(col)
cm = plt.cm.get_cmap(divergingcmap)
for c, p in zip(col, patches):
    plt.setp(p, 'facecolor', cm(c))

#plt.tight_layout();
"""if saving:
    plt.savefig(os.path.join(savefolder,file + '.pdf'), dpi=None, facecolor='w', edgecolor='w',
        transparent=False, bbox_inches='tight', pad_inches=0.05,
        metadata=None)"""



file = 'PosGU01Array01.00r0.5p3Array01.00r0.5p3Device1.2Drum1_15.5MHz105133' 
path = os.path.join(folder1,file)
df = pd.read_csv(os.path.join(path, 'Demod1.csv'), skiprows = 2) 
 
df['X um']=df['Green X']
df['Y um']= df['Green Y']
df.rnorm = df.r/df.r.max()        

print("Frequency in Hz: ",df.frequency[0])
# phase correction
df['phase_cor'] = df.phase + radians_correction(df.frequency[0]) ## correction for radians
df['phase_cor_wrap'] = df.phase_cor % (2 * np.pi) - 2 * np.pi # wrap to be between -2pi and 0

df.phasedeg_uncor = np.rad2deg(df.phase)
df.phasedeg = np.rad2deg(df.phase_cor_wrap)


plt.sca(ax2)
amp2 = df.pivot_table(index = 'Y um', columns = 'X um', values = 'r').sort_index(axis = 0, ascending = False) 
data = np.log10(amp2)
if rotating:
    data = rotatehack(data)
# set white at max, cut colorbar in half
vminax2 = np.array(data).min()
vmaxax2 = np.array(data).max()
greennorm = MidpointNormalize(vmin=vminax2, vmax=vmaxax2, midpoint=vmax)

if halfaxiscmap:
    axx, cbar = myheatmap(data,  norm = greennorm, return_cbar=True,
              cmap=cmapforhalf);
else:
    axx, cbar = myheatmap(data,  return_cbar=True,
              cmap=ampcmap);
plt.xlabel('')
plt.ylabel('')
ax3.axis('equal');
#topx = amp1.columns.max()
plt.xticks([topx-10, topx])
plt.gca().set_xticklabels([tenum, ''])
plt.yticks([]) # remove y ticks, since we can use the xticks alone on a square plot

# decrease number of ticks on colorbar
#cbar.ax.locator_params(nbins=2)
#cbar.set_ticks([-12,-16])
cbar.set_ticks([-7, -6,-5])
cbar.ax.set_yticklabels(['$10^{-7}$', '$10^{-6}$','$10^{-5}$']) ## make SURE this line is with the one above it
cbar3= cbar

if add_circle:
    if rotating:
        coordinateslist = coordlistrot
    else:
        coordinateslist = coordlist
    for coords in coordinateslist:        
        thiscircle = plt.Circle(coords, radius=radius, color='k', fill = None, alpha = 1, lw=0.5,)#linestyle='dashed')
        plt.gca().add_artist(thiscircle)

plt.sca(hiampax)
plt.errorbar(x=horiz, y = Ahivector, yerr = Ahierrvector,
             ecolor = 'k',
             markerfacecolor  = 'k',
             markeredgewidth  = 0,
             ls='None', marker = '.', 
             markersize = 5)
#             ecolor = [R1color,R2color,R3color],

hiampax = plt.gca()
hiampax.set_yscale('log')
#hiampax.set_xlabel ="Resonator number"
hiampax.set_ylabel="Amplitude (arb. units)"
hiampax.set_ylim(ymin=10**vminax2, ymax = 10**vmaxax2)

hiampax.annotate('$A_1$', (horiz[0],A1hi),color=R1color, va='center', xytext=(5, 0), textcoords='offset points',)
hiampax.annotate('$A_2$', (horiz[1],A2hi), color=R2color, va='center', xytext=(5, 0), textcoords='offset points',)
#hiampax.annotate('$A_3$', (horiz[2],A3hi), color=R3color, va='center', xytext=(5, 0), textcoords='offset points',)
sns.despine()        
        
plt.sca(ax4)
phase2 = df.pivot_table(index = 'Y um', columns = 'X um', values = 'phase_cor_wrap').sort_index(axis = 0, ascending = False) 
data = phase2
if rotating:
    data = rotatehack(data)
axx, cbar = myheatmap(data, return_cbar=True, vmin=-2*np.pi, vmax=0, cmap=divergingcmap);
plt.xlabel('')
plt.ylabel('')
ax4.axis('equal');
#topx = amp1.columns.max()
plt.xticks([topx-10, topx])
plt.gca().set_xticklabels([tenum, ''])
plt.yticks([]) # remove y ticks, since we can use the xticks alone on a square plot
cbar.ax.yaxis.set_ticks_position('left')
cbar.set_ticks([-2*np.pi,-np.pi,0])
cbar.ax.set_yticklabels(['$-$2\u03C0','$-$\u03C0','0'],ha='right')  # make sure this line is joined with the one above it.
cbar.ax.yaxis.set_tick_params(pad=1)  # your number may vary
cbar4 = cbar



for cbar in [cbar1, cbar2, cbar3, cbar4]:
    cbar.ax.tick_params(length=cbarticklength, width=tickwidth)


# add_sizebar(ax4, 10) # creates a scalebar a little bit covering the plot.

if add_circle:
    if rotating:
        coordinateslist = coordlistrot
    else:
        coordinateslist = coordlist
    for coords in coordinateslist:        
        thiscircle = plt.Circle(coords, radius=radius, color='k', fill = None, alpha = .8, lw=0.5,)#linestyle='dashed')
        plt.gca().add_artist(thiscircle)


mk = mk1550;
         
plt.sca(phist2) # Phase histogram 2
plt.hist(df.phasedeg[mk], bins=numbins,color=('k'),orientation='horizontal', 
         histtype='step', range=degreerange, linewidth=0.5); # outline
n, bins, patches = plt.hist(df.phasedeg[mk], bins=numbins,color=('#276419'),orientation='horizontal', range=degreerange);
#plt.ylabel('Phase (deg)')
#plt.xlabel('# pixels')
print ('Resonator 2 phase ' + str(df.phasedeg[mk][maskR2].mean()))
print ('Resonator 1 phase ' + str(df.phasedeg[mk][maskR1].mean()))
plt.yticks([df.phasedeg[mk][maskR2].mean(), 0,df.phasedeg[mk][maskR1].mean() ])
plt.ylim(degreerange)
ax = plt.gca()
ax.yaxis.set_label_position("right")
ax.yaxis.tick_right()
print("Phase difference (deg):")
print (df.phasedeg[mk][maskR1].mean() - df.phasedeg[mk][maskR2].mean())
## color bins, https://stackoverflow.com/questions/23061657/plot-histogram-with-colors-taken-from-colormap
bin_centers = 0.5 * (bins[:-1] + bins[1:])
bin_centers=includeendpoints(bin_centers)
# scale values to interval [0,1]
col = bin_centers - min(bin_centers)
col /= max(col)
cm = plt.cm.get_cmap(divergingcmap)
for c, p in zip(col, patches):
    plt.setp(p, 'facecolor', cm(c))
sns.despine(left=True, right=True, bottom=True)

hiampax.axis('off')
loampax.axis('off')

if includetitles:
    ax1.set_title(str(round(low_mode_MHz,2)) + " MHz")
    ax2.set_title(str(round(high_mode_MHz,2)) + " MHz")
    ax3.set_title(str(round(low_mode_MHz,2)) + " MHz")
    ax4.set_title(str(round(high_mode_MHz,2)) + " MHz")


for ax in [phist1, phist2]:
    ax.tick_params(direction='out', length=ticklength, width=tickwidth)
        
#plt.tight_layout();

bbox1 = ax1.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
heatmapsize = (bbox1.width, bbox1.height)


if saving:
    plt.savefig(os.path.join(savefolder, 'modemap_export.pdf'), dpi=None, facecolor='w',  edgecolor='w',
        transparent=False, bbox_inches='tight', pad_inches=0,
        metadata=None)
    print('File saved: ')
else:
    print('File not overwritten:')
print(os.path.join(savefolder, 'modemap_export_with_histograms.pdf'))

#saving = True
"""
## This stopped working but I already have the export.
add_circle= True

#plt.figure(figsize=(1,1))
plt.figure(figsize=(.7,.7))

plt.xlim(xlims)
plt.ylim(ylims)
plt.gca().axis('equal');
plt.xlabel('')
plt.ylabel('')
#topx = amp1.columns.max()
## 10 um measure-bar
#plt.xticks([topx-10, topx])
#plt.gca().set_xticklabels([tenum, ''])
## no x-ticks
plt.xticks([]) # no sizing
plt.yticks([]) # remove y ticks, since we can use the xticks alone on a square plot

if add_circle:
    if rotating:
        coordinateslist = coordlistrot
    else:
        coordinateslist = coordlist
    for coords in coordinateslist:
        thiscircle = plt.Circle(coords, radius=radius, color='k', fill = None, alpha = .8, lw=0.5,)#linestyle='dashed')
        plt.gca().add_artist(thiscircle)

for _,s in plt.gca().spines.items():
        s.set_linewidth(.5)
        
#plt.tight_layout();

#saving=True
if saving:
    plt.savefig(os.path.join(savefolder, 'circles_export.pdf'), dpi=None, facecolor='w',  edgecolor='w',
        transparent=False, bbox_inches='tight', pad_inches=0.005,
        metadata=None)
    print('File saved: ')
else:
    print('File not overwritten:')
print(os.path.join(savefolder, 'circles_export.pdf'))"""

sns.set_context('poster') # makes text larger again

In [ ]:
sns.set_context('paper') # makes text sized for paper

#saving = True
add_circle= True
rotating = True
#divergingcmap = 'twilight_shifted'
ampcmap = 'Greys' #'magma'
halfaxiscmap = False # slightly broken on the high frequency
includetitles = True

file = 'PosGU01Array01.00r0.5p3Array01.00r0.5p3Device1.2Drum_15.16MHz_1104325' 
path = os.path.join(folder1,file)
df = pd.read_csv(os.path.join(path, 'Demod1.csv'), skiprows = 2) 
#df.r = df.r/df.r.max() 
#print(df) 

df['X um']=df['Green X']
df['Y um']= df['Green Y']
# calculate corrected phase
df['phase_cor'] = df.phase + radians_correction(df.frequency[0]) ## correction for radians
df['phase_cor_wrap'] = df.phase_cor % (2 * np.pi) - 2 * np.pi # wrap to be between -2pi and 0

df.phasedeg_uncor = np.rad2deg(df.phase)
df.phasedeg = np.rad2deg(df.phase_cor_wrap)
df.rnorm = df.r/df.r.max()

widths = [2.6,.7,2.6,.7]
heights = [1, 1]
gs_kw = dict(width_ratios=widths, height_ratios=heights)

## (2,2) size.   #3.37, 2.8 is tight; 3.15 height gives space for scale bar.
sty={'figsize':(5.1,2.9), "constrained_layout":True, "gridspec_kw": gs_kw}

## (1,2) size
#sty={'figsize':(3.37,1.43), "constrained_layout":True}

fig, ((ax1, loampax, ax2, hiampax),(ax3, lowphaseax, ax4, hiphaseax)) = plt.subplots(2,4, **sty)

## label text for scalebar
tenum = u'\u200c          10 μm'

#plt.suptitle(file + '\n\n');
plt.sca(ax1) 

amp1 = df.pivot_table(index = 'Y um', columns = 'X um', values = 'r').sort_index(axis = 0, ascending = False) 
data = np.log10(amp1)
if rotating:
    data = rotatehack(data)
# set white at max, cut colorbar in half
vmin = np.array(data).min()
vmax = np.array(data).max()
greennorm = MidpointNormalize(vmin=vmin, vmax=vmax, midpoint=vmax)

if halfaxiscmap:
    axx,cbar = myheatmap(data,  norm = greennorm, return_cbar=True,
              cmap=cmapforhalf);
else:
    axx,cbar = myheatmap(data,   return_cbar=True,
              cmap=ampcmap);
plt.xlabel('')
plt.ylabel('')
ax1.axis('equal')
topx = data.columns.max()
plt.xticks([topx-10, topx])
plt.gca().set_xticklabels([tenum, ''])
plt.yticks([]) # remove y ticks, since we can use the xticks alone on a square plot
cbar1 = cbar

# decrease number of ticks on colorbar
"""tick_locator = mpl.ticker.MaxNLocator(nbins=4)
cbar.locator = tick_locator
cbar.update_ticks()"""
#cbar.set_ticks([-11, -16])
cbar.set_ticks([-7, -6,-5])
cbar.ax.set_yticklabels(['$10^{-7}$', '$10^{-6}$','$10^{-5}$']) ## make SURE this line is with the one above it
ylims = plt.ylim()
xlims = plt.xlim()

if add_circle:
    if rotating:
        coordinateslist = coordlistrot
    else:
        coordinateslist = coordlist
    for coords in coordinateslist:
        thiscircle = plt.Circle(coords, radius=radius, color='k', fill = None, alpha = 1, lw=0.5,)#linestyle='dashed')
        plt.gca().add_artist(thiscircle)
        
plt.sca(loampax)        
plt.errorbar(x=horiz, y = Alowvector, yerr = Alowerrvector,
             ecolor = 'k',
             markerfacecolor  = 'k',
             markeredgewidth  = 0,
             ls='None', marker = '.', 
             markersize = 5)
#             ecolor = [R1color,R2color,R3color],

loampax = plt.gca()
loampax.set_yscale('log')
#loampax.set_xlabel ="Resonator number"
loampax.set_ylabel="Amplitude (arb. units)"
loampax.set_ylim(ymin=10**vmin, ymax = 10**vmax)
#loampax.axis('off')

loampax.annotate('$A_1$', (horiz[0],A1low),color=R1color, va='center', xytext=(5, -2), textcoords='offset points',)
loampax.annotate('$A_2$', (horiz[1],A2low), color=R2color, va='center', xytext=(5, 2), textcoords='offset points',)
loampax.annotate('$A_3$', (horiz[2],A3low), color=R3color, va='center', xytext=(5, 0), textcoords='offset points',)
sns.despine()

plt.sca(ax3) 
phase1 = df.pivot_table(index = 'Y um', columns = 'X um', values = 'phase_cor_wrap').sort_index(axis = 0, ascending = False) 
data = phase1
if rotating:
    data = rotatehack(data)
axx, cbar = myheatmap(data, return_cbar=True,  vmin=-2*np.pi, vmax=0, cmap=divergingcmap);
ax2.axis('equal');
plt.xlabel('')
plt.ylabel('')
#topx = amp1.columns.max()
plt.xticks([topx-10, topx])
plt.gca().set_xticklabels([tenum, ''])
cbar.ax.yaxis.set_ticks_position('left')
cbar.set_ticks([-2*np.pi,-np.pi,0])
cbar.ax.set_yticklabels(['$-$2\u03C0','$-$\u03C0','0'],ha='right')  # make sure this line is joined with the one above it.
cbar.ax.yaxis.set_tick_params(pad=1)  # your number may vary
plt.yticks([]) # remove y ticks, since we can use the xticks alone on a square plot
cbar2 = cbar

if add_circle:
    if rotating:
        coordinateslist = coordlistrot
    else:
        coordinateslist = coordlist
    for coords in coordinateslist:
        thiscircle = plt.Circle(coords, radius=radius, color='k', fill = None, alpha = .8, lw=0.5,)#linestyle='dashed')
        plt.gca().add_artist(thiscircle)
        
plt.sca(lowphaseax) # Phase plot, low freq mode
plt.errorbar(x=horiz, y = dlowvector, yerr = dlowerrvector,
             ecolor = 'k',
             markerfacecolor  = 'k',
             markeredgewidth  = 0,
             ls='None', marker = '.', 
             markersize = 5)
#             ecolor = [R1color,R2color,R3color],
#             ax = lowphaseax,

#lowphaseax.yaxis.set_major_locator(plt.MultipleLocator(90))
#lowphaseax.set_yscale('log')
#lowphaseax.set_xlabel ="Resonator number"
#lowphaseax.set_ylabel="Phase (deg)"
lowphaseax.set_ylim(degreerange)

lowphaseax.annotate('$\delta_1$', (horiz[0],d1low),color=R1color, va='center', xytext=(5, 2), textcoords='offset points',)
lowphaseax.annotate('$\delta_2$', (horiz[1],d2low), color=R2color, va='center', xytext=(5, -2), textcoords='offset points',)
lowphaseax.annotate('$\delta_3$', (horiz[2],d3low), color=R3color, va='center', xytext=(5, 0), textcoords='offset points',)
#sns.despine()


#plt.tight_layout();
"""if saving:
    plt.savefig(os.path.join(savefolder,file + '.pdf'), dpi=None, facecolor='w', edgecolor='w',
        transparent=False, bbox_inches='tight', pad_inches=0.05,
        metadata=None)"""



file = 'PosGU01Array01.00r0.5p3Array01.00r0.5p3Device1.2Drum1_15.5MHz105133' 
path = os.path.join(folder1,file)
df = pd.read_csv(os.path.join(path, 'Demod1.csv'), skiprows = 2) 
 
df['X um']=df['Green X']
df['Y um']= df['Green Y']
df.rnorm = df.r/df.r.max()        

print("Frequency in Hz: ",df.frequency[0])
# phase correction
df['phase_cor'] = df.phase + radians_correction(df.frequency[0]) ## correction for radians
df['phase_cor_wrap'] = df.phase_cor % (2 * np.pi) - 2 * np.pi # wrap to be between -2pi and 0

df.phasedeg_uncor = np.rad2deg(df.phase)
df.phasedeg = np.rad2deg(df.phase_cor_wrap)


plt.sca(ax2)
amp2 = df.pivot_table(index = 'Y um', columns = 'X um', values = 'r').sort_index(axis = 0, ascending = False) 
data = np.log10(amp2)
if rotating:
    data = rotatehack(data)
# set white at max, cut colorbar in half
vminax2 = np.array(data).min()
vmaxax2 = np.array(data).max()
greennorm = MidpointNormalize(vmin=vminax2, vmax=vmaxax2, midpoint=vmax)

if halfaxiscmap:
    axx, cbar = myheatmap(data,  norm = greennorm, return_cbar=True,
              cmap=cmapforhalf);
else:
    axx, cbar = myheatmap(data,  return_cbar=True,
              cmap=ampcmap);
plt.xlabel('')
plt.ylabel('')
ax3.axis('equal');
#topx = amp1.columns.max()
plt.xticks([topx-10, topx])
plt.gca().set_xticklabels([tenum, ''])
plt.yticks([]) # remove y ticks, since we can use the xticks alone on a square plot

# decrease number of ticks on colorbar
#cbar.ax.locator_params(nbins=2)
#cbar.set_ticks([-12,-16])
cbar.set_ticks([-7, -6,-5])
cbar.ax.set_yticklabels(['$10^{-7}$', '$10^{-6}$','$10^{-5}$']) ## make SURE this line is with the one above it
cbar3= cbar

if add_circle:
    if rotating:
        coordinateslist = coordlistrot
    else:
        coordinateslist = coordlist
    for coords in coordinateslist:        
        thiscircle = plt.Circle(coords, radius=radius, color='k', fill = None, alpha = 1, lw=0.5,)#linestyle='dashed')
        plt.gca().add_artist(thiscircle)

plt.sca(hiampax)
plt.errorbar(x=horiz, y = Ahivector, yerr = Ahierrvector,
             ecolor = 'k',
             markerfacecolor  = 'k',
             markeredgewidth  = 0,
             ls='None', marker = '.', 
             markersize = 5)
#             ecolor = [R1color,R2color,R3color],

hiampax = plt.gca()
hiampax.set_yscale('log')
#hiampax.set_xlabel ="Resonator number"
hiampax.set_ylabel="Amplitude (arb. units)"
hiampax.set_ylim(ymin=10**vminax2, ymax = 10**vmaxax2)

hiampax.annotate('$A_1$', (horiz[0],A1hi),color=R1color, va='center', xytext=(5, 0), textcoords='offset points',)
hiampax.annotate('$A_2$', (horiz[1],A2hi), color=R2color, va='center', xytext=(5, 0), textcoords='offset points',)
#hiampax.annotate('$A_3$', (horiz[2],A3hi), color=R3color, va='center', xytext=(5, 0), textcoords='offset points',)
sns.despine()        
        
plt.sca(ax4)
phase2 = df.pivot_table(index = 'Y um', columns = 'X um', values = 'phase_cor_wrap').sort_index(axis = 0, ascending = False) 
data = phase2
if rotating:
    data = rotatehack(data)
axx, cbar = myheatmap(data, return_cbar=True, vmin=-2*np.pi, vmax=0, cmap=divergingcmap);
plt.xlabel('')
plt.ylabel('')
ax4.axis('equal');
#topx = amp1.columns.max()
plt.xticks([topx-10, topx])
plt.gca().set_xticklabels([tenum, ''])
plt.yticks([]) # remove y ticks, since we can use the xticks alone on a square plot
cbar.ax.yaxis.set_ticks_position('left')
cbar.set_ticks([-2*np.pi,-np.pi,0])
cbar.ax.set_yticklabels(['$-$2\u03C0','$-$\u03C0','0'],ha='right')  # make sure this line is joined with the one above it.
cbar.ax.yaxis.set_tick_params(pad=1)  # your number may vary
cbar4 = cbar



for cbar in [cbar1, cbar2, cbar3, cbar4]:
    cbar.ax.tick_params(length=cbarticklength, width=tickwidth)


# add_sizebar(ax4, 10) # creates a scalebar a little bit covering the plot.

if add_circle:
    if rotating:
        coordinateslist = coordlistrot
    else:
        coordinateslist = coordlist
    for coords in coordinateslist:        
        thiscircle = plt.Circle(coords, radius=radius, color='k', fill = None, alpha = .8, lw=0.5,)#linestyle='dashed')
        plt.gca().add_artist(thiscircle)


mk = mk1550;
         
plt.sca(hiphaseax) # Phase plot, high freq
plt.errorbar(x=horiz, y = dhivector, yerr = dhierrvector,
             ecolor = 'k',
             markerfacecolor  = 'k',
             markeredgewidth  = 0,
             ls='None', marker = '.', 
             markersize = 5)
#             ecolor = [R1color,R2color,R3color],

hiphaseax = plt.gca()
hiphaseax.yaxis.set_major_locator(plt.MultipleLocator(90))
#hiphaseax.set_yscale('log')
#hiphaseax.set_xlabel ="Resonator number"
hiphaseax.set_ylabel="Amplitude (arb. units)"
if wrapping:
    hiphaseax.set_ylim(degreerange)

hiphaseax.annotate('$\delta_1$', (horiz[0],d1hi),color=R1color, va='center', xytext=(5, 0), textcoords='offset points',)
hiphaseax.annotate('$\delta_2$', (horiz[1],d2hi), color=R2color, va='center', xytext=(5, 0), textcoords='offset points',)
hiphaseax.annotate('$\delta_3$', (horiz[2],d3hi), color=R3color, va='center', xytext=(5, 0), textcoords='offset points',)


hiampax.axis('off')
loampax.axis('off')
lowphaseax.axis('off')
hiphaseax.axis('off')

if includetitles:
    ax1.set_title(str(round(low_mode_MHz,2)) + " MHz")
    ax2.set_title(str(round(high_mode_MHz,2)) + " MHz")
    ax3.set_title(str(round(low_mode_MHz,2)) + " MHz")
    ax4.set_title(str(round(high_mode_MHz,2)) + " MHz")
        
#plt.tight_layout();

bbox1 = ax1.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
heatmapsize = (bbox1.width, bbox1.height)


if saving:
    plt.savefig(os.path.join(savefolder, 'modemap_export.pdf'), dpi=None, facecolor='w',  edgecolor='w',
        transparent=False, bbox_inches='tight', pad_inches=0,
        metadata=None)
    print('File saved: ')
else:
    print('File not overwritten:')
print(os.path.join(savefolder, 'modemap_export.pdf'))

sns.set_context('poster') # makes text larger again

In [ ]:
## ~~~~~~~ See above for the plots
display(heatmapsize)

avgsize = sum(list(heatmapsize))/2

In [ ]:
## single resonator

file = 'PosGU01Arrray01.00r0.5p3Device0.0Drum210V_15.6MHz100338' 
path = os.path.join(folder2,file)
df = pd.read_csv(os.path.join(path, 'Demod1.csv'), skiprows = 2) 

df['X um']=df['Green X']
df['Y um']= df['Green Y']
# calculate corrected phase
df['phase_cor'] = df.phase + radians_correction(df.frequency[0]) ## correction for radians
df['phase_cor_wrap'] = df.phase_cor % (2 * np.pi) - 2 * np.pi # wrap to be between -2pi and 0

plt.figure()
amp1 = df.pivot_table(index = 'Y um', columns = 'X um', values = 'r').sort_index(axis = 0, ascending = False)

singleresdata = (amp1)/(amp1.max().max())
max2 = amp1.max().max()
# set white at max, cut colorbar in half
vmin = np.array(singleresdata).min()
vmax = np.array(singleresdata).max()
greennorm = MidpointNormalize(vmin=vmin, vmax=vmax, midpoint=vmax)

ax, cbar = myheatmap(singleresdata, 'amplitude', norm = greennorm,
                  cmap=cmapforhalf, return_cbar=True);
plt.xlabel(u'$x$ [μm]')
plt.ylabel(u'$y$ [μm]')
ax.axis('equal')
#cbar.set_ticks([-16,-12])

fig, (ax1, ax2) = plt.subplots(1, 2,  figsize=(13.23,5))

#plt.suptitle(file + '\n\n');

plt.sca(ax1)
amp1 = df.pivot_table(index = 'Y um', columns = 'X um', values = 'r').sort_index(axis = 0, ascending = False) 
data = np.log(amp1)
# set white at max, cut colorbar in half
vmin = np.array(data).min()
vmax = np.array(data).max()
greennorm = MidpointNormalize(vmin=vmin, vmax=vmax, midpoint=vmax)

_, cbar1 = myheatmap(data, 'log amplitude', norm = greennorm,
          cmap=cmapforhalf, return_cbar=True);
plt.xlabel(u'$x$ [μm]')
plt.ylabel(u'$y$ [μm]')
ax1.axis('equal')
cbar1.set_ticks([-16,-12])



plt.sca(ax2) 
phase1 = df.pivot_table(index = 'Y um', columns = 'X um', values = 'phase_cor_wrap').sort_index(axis = 0, ascending = False) 
_, cbar = myheatmap(phase1, 'phase',vmin=-2*np.pi, vmax=0, cmap=divergingcmap, return_cbar=True);
ax2.axis('equal');
plt.xlabel(u'$x$ [μm]')
plt.ylabel(u'$y$ [μm]')
cbar.set_ticks([-2*np.pi,-np.pi,0])
cbar.ax.set_yticklabels(['$-$2\u03C0','$-$\u03C0','0'],ha='right')  # make sure this line is joined with the one above it.
cbar.ax.yaxis.set_tick_params(pad=40)  # your number may vary

print(file)
plt.tight_layout();

In [ ]:
ax = myheatmap(amp1.iloc[6:-2,5:-5])
ax.axis('equal')

In [ ]:
# need to fix
file = 'PosGU01Arrray01.00r0.5p3Device0.1Drum2_10V114238' 
path = os.path.join(folder2,file)
df = pd.read_csv(os.path.join(path, 'Demod1.csv'), skiprows = 2) 
df['X um']=df['Green X']
df['Y um']= df['Green Y']
print(df.frequency[0])
# calculate corre"cted phase
df['phase_cor'] = df.phase + radians_correction(df.frequency[0]) ## correction for radians
df['phase_cor_wrap'] = df.phase_cor % (2 * np.pi) - 2 * np.pi # wrap to be between -2pi and 0

fig, (ax1, ax2) = plt.subplots(1, 2,  figsize=(13.23,5))

#plt.suptitle(file + '\n\n');

plt.sca(ax1)
amp1 = df.pivot_table(index = 'Y um', columns = 'X um', values = 'r').sort_index(axis = 0, ascending = False) 
data = np.log(amp1)
# set white at max, cut colorbar in half
vmin = np.array(data).min()
vmax = np.array(data).max()
greennorm = MidpointNormalize(vmin=vmin, vmax=vmax, midpoint=vmax)

myheatmap(data, 'log amplitude', norm = greennorm,
          cmap=cmapforhalf);
plt.xlabel(u'$x$ [μm]')
plt.ylabel(u'$y$ [μm]')
ax1.axis('equal')


plt.sca(ax2) 
phase1 = df.pivot_table(index = 'Y um', columns = 'X um', values = 'phase_cor_wrap').sort_index(axis = 0, ascending = False) 
_,cbar = myheatmap(phase1, 'phase',vmin=-2*np.pi, vmax=0, cmap=divergingcmap, return_cbar=True);
ax2.axis('equal');
plt.xlabel(u'$x$ [μm]')
plt.ylabel(u'$y$ [μm]')
cbar.set_ticks([-2*np.pi,-np.pi,0])
cbar.ax.set_yticklabels(['$-$2\u03C0','$-$\u03C0','0'],ha='right')  # make sure this line is joined with the one above it.
"""cbar.set_ticks([-np.pi, 0, np.pi])
cbar.ax.set_yticklabels(['$-$\u03C0', '0', '$+$\u03C0'],ha='right')  # make sure this line is joined with the one above it."""
cbar.ax.yaxis.set_tick_params(pad=40)  # your number may vary

print(file)
plt.tight_layout();

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2,  figsize=(13.23,5))

#plt.suptitle(file + '\n\n');

plt.sca(ax1)
amp1 = df.pivot_table(index = 'Y um', columns = 'X um', values = 'r').sort_index(axis = 0, ascending = False) 
data = np.log(amp1)
# set white at max, cut colorbar in half
vmin = np.array(data).min()
vmax = np.array(data).max()
greennorm = MidpointNormalize(vmin=vmin, vmax=vmax, midpoint=vmax)

myheatmap(data, 'log amplitude', norm = greennorm,
          cmap=cmapforhalf);
plt.xlabel(u'$x$ [μm]')
plt.ylabel(u'$y$ [μm]')
ax1.axis('equal')


plt.sca(ax2) 
phase1 = df.pivot_table(index = 'Y um', columns = 'X um', values = 'phase_cor_wrap').sort_index(axis = 0, ascending = False) 
myheatmap(phase1, 'phase',vmin=-2*np.pi, vmax=0, cmap=divergingcmap);
ax2.axis('equal');
plt.xlabel(u'$x$ [μm]')
plt.ylabel(u'$y$ [μm]')

print(file)
plt.tight_layout();

In [ ]:
x1 = df.pivot_table(index = 'Y um', columns = 'X um', values = 'x').sort_index(axis = 0, ascending = False) 

myheatmap(x1, 'x', 
          cmap='viridis');
plt.xlabel(u'$x$ [μm]')
plt.ylabel(u'$y$ [μm]')
plt.gca().axis('equal')

plt.figure()
myheatmap(-np.log(-x1), '-log -x', 
          cmap='viridis');
plt.xlabel(u'$x$ [μm]')
plt.ylabel(u'$y$ [μm]')
plt.gca().axis('equal')

plt.figure()
myheatmap(np.log(x1), 'log x', 
          cmap='viridis');
plt.xlabel(u'$x$ [μm]')
plt.ylabel(u'$y$ [μm]')
plt.gca().axis('equal')

"""plt.figure()
myheatmap(np.nanmax(np.array(-np.log(-x1)),np.array(np.log(x1)))"""

In [ ]:
#np.nanmax(np.array(-np.log(-x1)),np.array(np.log(x1)))

In [ ]:
  
# set midpoint at zero.
vmin = np.array(phase1).min()
vmax = np.array(phase1).max()
norm = MidpointNormalize(vmin=vmin, vmax=vmax, midpoint=0)


## Plot with a norm at 0
myheatmap(phase1, 'phase_cor_wrap', norm = norm,
          cmap=divergingcmap);
plt.gca().axis('equal');
plt.xlabel(u'$x$ [μm]')
plt.ylabel(u'$y$ [μm]');

In [ ]:
if plot3d:
    from mpl_toolkits import mplot3d
    ax = plt.axes(projection='3d')
    X, Y = np.meshgrid(phase1.columns,phase1.index )
    ax.plot_surface(X,Y,phase1, 
                    norm=norm,  # set midpoint at 0.
                    antialiased=False,
                    cmap=divergingcmap )
    plt.xlabel(phase1.columns.name)
    plt.ylabel(phase1.index.name)
    ax.set_title(file);
    ax.view_init(elev=60., azim=0)

In [ ]:
## Cut colorbar in half.

data = np.log(amp1)

# set white at max.
vmin = np.array(data).min()
vmax = np.array(data).max()
greennorm = MidpointNormalize(vmin=vmin, vmax=vmax, midpoint=vmax)


## Plot with a midpoint at 0.
myheatmap(data, 'log amplitude', norm = greennorm,
          cmap=cmapforhalf);
plt.gca().axis('equal');
plt.xlabel(u'$x$ [μm]')
plt.ylabel(u'$y$ [μm]');

In [ ]:
myheatmap(np.log(amp1), 'log amplitude', cmap = 'viridis');
plt.xlabel(u'$x$ [μm]')
plt.ylabel(u'$y$ [μm]')
plt.gca().axis('equal')

if plot3d:
    plt.figure()
    ax = plt.axes(projection='3d')
    X, Y = np.meshgrid(amp1.columns,amp1.index )
    ax.plot_surface(X,Y,(amp1), 
                    antialiased=False,
                    cmap='viridis', )
    plt.xlabel(amp1.columns.name)
    plt.ylabel(amp1.index.name)
    ax.set_title(file);
    ax.view_init(elev=60., azim=0)

In [ ]:
# This import registers the 3D projection, but is otherwise unused.
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import

import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import numpy as np


fig = plt.figure()
ax = fig.gca(projection='3d')

# Make data.
X = np.arange(-5, 5, 0.25)
Y = np.arange(-5, 5, 0.25)
X, Y = np.meshgrid(X, Y)
R = np.sqrt(X**2 + Y**2)
Z = np.sin(R)

# Plot the surface.
surf = ax.plot_surface(X, Y, Z, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)

# Customize the z axis.
ax.set_zlim(-1.01, 1.01)
ax.zaxis.set_major_locator(LinearLocator(10))
ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

# Add a color bar which maps values to colors.
fig.colorbar(surf, shrink=0.5, aspect=5)

plt.show()

In [ ]:
#### Want to plot together: single resonance peak, mode map, double resonance peak, mode map
import lmfit as lf
from lmfit.models import DampedOscillatorModel

widths = [1,1]
heights = [1, 1]
gs_kw = dict(width_ratios=widths, height_ratios=heights)

sns.set_context('paper')
# subplotssty={'figsize':(3.37,3.3), "constrained_layout":True, 'gridspec_kw':gs_kw} # use 3.3 if no plot title
subplotssty={'figsize':(3.37,3.2), "constrained_layout":True, 'gridspec_kw':gs_kw}

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2, **subplotssty)

"""Plot single resonator curve"""

## Mac vs Windows
#base = '/Volumes/Aleman-Lab/Group/Projects/GrapheneCoupledResonators/Data/Sample2/ResonanceCurves/'
#savefolder = '/Volumes/Aleman-Lab/Group/Papers and Abstracts/Coupled Resonator Spatial Imaging/Figures/python_exports/'
base = r'\\cas-fs1.uoregon.edu\Material-Science-Institute\Aleman-Lab\Group\Projects\GrapheneCoupledResonators\Data\Sample2\ResonanceCurves'
savefolder = r'\\cas-fs1.uoregon.edu\Material-Science-Institute\Aleman-Lab\Group\Papers and Abstracts\Coupled Resonator Spatial Imaging\Figures\python_exports'
path = os.path.join(base, 'Array01.00r0.5p3Device0.0Drum1_2','Data.csv')
colnames = ['SweepVal', 'F', 'R', 'theta', 'NEP_BW', 'R_ref', 'theta_ref', 'Amp', 'Phase']
df = pd.read_csv(path, names = colnames)

#Find resonant frequency and normalize F and R^2 values
maxR = df['R'].max()
maxFreqIn = (df['R'].idxmax())
max1 = maxR

df['Rnorm'] = (df.R) / (maxR)
df['Fnorm'] = df.F / 10**6
res = df['Fnorm'].values[maxFreqIn]
# Define model
model = DampedOscillatorModel()
model.param_names
# Fit
guessQ = 1000
params = model.make_params(sigma = 1/(2*guessQ), center = res, amplitude = 1 / (guessQ*res))
result = model.fit(df.Rnorm, params, x = df.Fnorm)
# Plot
sty = {'markersize':4, "lw":1}
ax1.plot(df.Fnorm, df.Rnorm, '.', color='#276419' , **sty )
ax1.plot(df.Fnorm, result.best_fit, '-k', **sty)
ax1.set_xlabel('Frequency (MHz)')
ax1.set_ylabel('Amplitude')

""" Plot the mode map for the single resonator """
plt.sca(ax2)
data = singleresdata.iloc[6:-2,4:-4]
if rotating:
    data=rotatehack(data)
#vmin = np.array(data).min()
vmax = np.array(data).max()
greennorm = MidpointNormalize(vmin=0, vmax=vmax, midpoint=vmax)
_, cbar = myheatmap(data, 'Amplitude', norm = greennorm,
                      cmap=cmapforhalf, return_cbar=True);
#cbar.ax.set_ylim(-1,1)
cbar.set_ticks([1,0])
cbar.ax.tick_params(length=cbarticklength, width=tickwidth)
ax2.axis('equal')

# Create scale-bar
topx = data.columns.max()
plt.xticks([topx-10, topx])
plt.gca().set_xticklabels([u'            10 μm', ''])
plt.yticks([]) # remove y ticks, since we can use the xticks alone on a square plot

#plt.xlabel(u'$x$ [μm]')
#plt.ylabel(u'$y$ [μm]')
plt.xlabel('')
plt.ylabel('')

""" Double resonant curve """

plt.sca(ax3)
#base = '/Volumes/Aleman-Lab/Group/Projects/GrapheneCoupledResonators/Data/Sample2/ResonanceCurves/'
base = r'\\cas-fs1.uoregon.edu\Material-Science-Institute\Aleman-Lab\Group\Projects\GrapheneCoupledResonators\Data\Sample2\ResonanceCurves'
path = os.path.join(base, 'Array01.00r0.5p3Device1.2Drum1Couping500/Data.csv')
colnames = ['SweepVal', 'F', 'R', 'theta', 'NEP_BW', 'R_ref', 'theta_ref', 'Amp', 'Phase']
df = pd.read_csv(path, names = colnames)
maxR = df['R'].max()
df['Rnorm'] = (df.R) / (maxR)
max3 = maxR
df['Fnorm'] = df.F / 10**6
sty = {'markersize':4}
ax3.plot(df.Fnorm, (df.Rnorm), '.', color=('#276419'),**sty)
ax3.set_xlabel('Frequency (MHz)')
ax3.set_ylabel('Amplitude')
## add marks where the frequencies are
axtop = ax3.twiny()
xlims = ax3.get_xlim()
axtop.set_xlim(xlims)
axtop.tick_params(bottom=False, top=True, left=False, right=False)
axtop.set_xticks([low_mode_Hz / 1e6, high_mode_Hz / 1e6])
#axtop.set_xticklabels([])
axtop.set_xticklabels(['$f_s$', '$f_a$'])


""" Mode map corresponding to double resonant curve """

plt.sca(ax4)
max4 = doubleresonatordata.max().max()
data = doubleresonatordata/doubleresonatordata.max().max()
if rotating:
    data = rotatehack(data)
#vmin = np.array(data).min()
vmax = np.array(data).max()
greennorm = MidpointNormalize(vmin=0, vmax=vmax, midpoint=vmax)

_,cbar = myheatmap(data, 'Amplitude', norm = greennorm,
          cmap=cmapforhalf, return_cbar= True);
ax4.axis('equal')
cbar.set_ticks([1,0])
cbar.ax.tick_params(length=cbarticklength, width=tickwidth)


if add_circle:
    if rotating:
        coordinateslist = coordlistrot
    else:
        coordinateslist = coordlist
    for coords in coordinateslist:  
        thiscircle = plt.Circle(coords, radius=radius, color='w', fill = None, alpha = .5, lw = .5)
        plt.gca().add_artist(thiscircle)
        
plt.title('$f_s$')

# Create scale-bar
topx = data.columns.max()
plt.xticks([topx-10, topx])
plt.gca().set_xticklabels([u'           10 μm', ''])
plt.yticks([]) # remove y ticks, since we can use the xticks alone on a square plot

#plt.xlabel(u'$x$ [μm]')
#plt.ylabel(u'$y$ [μm]')
plt.xlabel('')
plt.ylabel('')

for ax in [ ax3, ax1]:
    ax.tick_params(direction='out', length=ticklength, width=tickwidth)
    ax.set_yticks([0,1])
"""    ax.spines['left'].set_linewidth(.5)
    ax.spines['right'].set_linewidth(.5)
    ax.spines['top'].set_linewidth(.5)
    ax.spines['bottom'].set_linewidth(.5)"""
    
sns.despine()
    
savename = 'ResCurvesAndModeMaps.pdf'
if saving:
    plt.savefig(os.path.join(savefolder,savename), bbox_inches='tight',pad_inches=0)
    print('Figure saved:')
else:
    print("Figure not overwritten:")
print(os.path.join(savefolder, savename))

sns.set_context('poster') # make font large again.

In [ ]:
print(max1)
print(max2)
print(max3)
print(max4)